# 📚 CourseConnect CLI API — Interactive Course Management System

📖 Description

A professional-grade, console-based Course Management System built with Python and Rich, designed for educational platforms. This interactive CLI allows administrators to assign students to courses, manage enrollments, track progress, and view statuses — all with elegant terminal tables and user-friendly prompts.

Ideal for developers and educators seeking a hands-on, text-based alternative to web dashboards, this notebook demonstrates how to build clean, modular, and interactive APIs for academic management.

# 🚀 **2. Setting Up the Environment**

## **2.1 Connect to google colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change directory
import os
os.chdir('/content/drive/MyDrive/Sprints task/Graduation Project')

## 2.2 Import Necessary libraries

In [ ]:
!pip install mysql-connector-python
!pip install rich colorama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 22.0 MB/s eta 0:00:00


In [ ]:
# 🔹 Standard Library Imports
import os
import re
import shutil
import mysql.connector
import sqlite3
import string
import time
import random
import warnings
import datetime
from tqdm import tqdm
import sys

# 🔹 Data Science & Visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import graphviz
import json

# 🔹 CLI & Table Formatting
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.theme import Theme
from rich.box import SIMPLE
from rich import box
from tabulate import tabulate
from prettytable import PrettyTable
from rich.table import Table as RichTable
from rich.text import Text
from rich.markdown import Markdown
from rich.align import Align
from rich.prompt import Prompt, Confirm
import uuid
from mysql.connector import IntegrityError
from rich.console import Console
from rich.table import Table
from rich import box
from rich.text import Text
from colorama import Fore

import google.generativeai as genai
from uuid import uuid4

# 🔹 Initialize Rich Console
console = Console()

# Connect to Databse

In [ ]:
# Connection details
HOST = 'db.cl0oa2kwiw65.me-central-1.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'bo13QknuzQ5nKR95QU9i'
DB_NAME = 'CourseDataBase'

In [ ]:
def connect_to_mysql(host, user, password, database=None):
    try:
        # Connect to the MySQL database
        db_conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Check if connection is successful
        if db_conn.is_connected():
            console.print(f"✅ Successfully connected to the MySQL database!", style="bold green")
            return db_conn
        else:
            console.print(f"❌ Failed to connect to the database.", style="bold red")
            return None
    except mysql.connector.Error as err:
        console.print(f"❌ Error: {err}", style="bold red")
        return None

In [ ]:
db_conn = connect_to_mysql(
    host=HOST,
    user=USER,
    password=PASSWORD
)

✅ Successfully connected to the MySQL database!

In [ ]:
cursor = db_conn.cursor()

In [ ]:
def connect_to_mysql(host=HOST, user=USER, password=PASSWORD, database=DB_NAME):
    try:
        # Connect to the MySQL database
        db_conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database # Database is specified here
        )

        # Check if connection is successful
        if db_conn.is_connected():
            console.print(f"✅ Successfully connected to the MySQL database!", style="bold green")
            # Explicitly select the database
            db_conn.database = database # Added line to select the database
            return db_conn
        else:
            console.print(f"❌ Failed to connect to the database.", style="bold red")
            return None
    except mysql.connector.Error as err:
        console.print(f"❌ Error: {err}", style="bold red")
        return None

In [ ]:
db_conn = connect_to_mysql()

✅ Successfully connected to the MySQL database!

In [ ]:
cursor = db_conn.cursor()

# **2. Users Table**

## **2.1 Add User**

In [ ]:
def add_users(cursor):
    console = Console()
    console.print("[bold cyan]👤 Add New User[/bold cyan]\n")

    while True:
        user_id = str(uuid.uuid4())
        full_name = Prompt.ask("👤 Full Name")
        email = Prompt.ask("📧 Email")
        password_hash = Prompt.ask("🔑 Password Hash", password=True)

        role = Prompt.ask("🎭 Role", choices=["student", "instructor", "admin"], default="student")

        bio = Prompt.ask("🧾 Bio", default="")
        linkedin_url = Prompt.ask("🔗 LinkedIn URL", default="")
        github_url = Prompt.ask("💻 GitHub URL", default="")
        profile_picture = Prompt.ask("🖼️ Profile Picture URL", default="")

        # Get preferences as a string
        preferences_str = Prompt.ask("⚙️ Preferences (JSON)", default="{}")

        # Attempt to parse the string as JSON, falling back to an empty dictionary if invalid
        try:
            preferences = json.loads(preferences_str)
        except json.JSONDecodeError:
            preferences = {}  # Use an empty dictionary if parsing fails
            console.print("[bold yellow]⚠️ Invalid JSON format for preferences. Using an empty dictionary.[/bold yellow]")

        education_level = Prompt.ask("🎓 Education Level", default="")
        years_of_experience = Prompt.ask("📅 Years of Experience", default="0")
        upload_cv_url = Prompt.ask("📄 Upload CV URL", default="")
        preferred_learning_platform = Prompt.ask("📚 Preferred Learning Platform", default="")
        location = Prompt.ask("📍 Location", default="")
        available_for_projects = Confirm.ask("🛠️ Available for Projects?", default=False)
        programming_languages = Prompt.ask("👨‍💻 Programming Languages", default="")
        tools_and_technologies = Prompt.ask("🧰 Tools & Technologies", default="")
        interests = Prompt.ask("🎯 Interests", default="")
        preferred_project_types = Prompt.ask("🧪 Preferred Project Types", default="")

        now = datetime.datetime.now()

        cursor.execute("""
            INSERT INTO users (
                user_ID, full_name, email, password_hash, role, bio, linkedin_url,
                github_url, profile_picture, preferences, education_level, years_of_experience,
                upload_cv_url, preferred_learning_platform, location, available_for_projects,
                programming_languages, tools_and_technologies, interests, preferred_project_types,
                created_at, updated_at
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
        """, (
            user_id, full_name, email, password_hash, role, bio, linkedin_url,
            github_url, profile_picture, json.dumps(preferences), education_level, int(years_of_experience), # Convert preferences to JSON string
            upload_cv_url, preferred_learning_platform, location, available_for_projects,
            programming_languages, tools_and_technologies, interests, preferred_project_types,
            now, now
        ))

        console.print(f"[green]✅ User [bold]{full_name}[/bold] added successfully![/green]")

        if not Confirm.ask("\n➕ Add another user?", default=False):
            break

In [ ]:
add_users(cursor)

👤 Add New User

👤 Full Name:

Mohamed Hatem


📧 Email:

mohamedhatem@gmail.com


...

🧪 Preferred Project Types ():

Nan


✅ User Mohamed Amr added successfully!

➕ Add another user? [y/n] (n):

n


In [ ]:
# Add temo user which we will delete
add_users(cursor)

👤 Add New User

👤 Full Name:

Test User for delete


📧 Email:

Nan


...

🧪 Preferred Project Types ():

Nan


✅ User Test User for delete added successfully!

➕ Add another user? [y/n] (n):

n


## **2.2 Delete Users**

In [ ]:
def delete_user(cursor):
    console = Console()
    console.print("[bold red]🗑️ Delete a User[/bold red]\n")

    # Fetch all users
    cursor.execute("SELECT user_ID, full_name, email, role FROM users")
    users = cursor.fetchall()

    if not users:
        console.print("[bold yellow]⚠️ No users found in the database.[/bold yellow]")
        return

    # Display users in a table
    table = Table(title="👥 List of Users", show_lines=True)
    table.add_column("No.", style="dim", width=6)
    table.add_column("Full Name", style="cyan")
    table.add_column("Email", style="magenta")
    table.add_column("Role", style="green")

    for i, user in enumerate(users, start=1):
        user_id, full_name, email, role = user
        table.add_row(str(i), full_name, email, role)

    console.print(table)

    # Ask which user to delete
    user_index = int(Prompt.ask("Enter the number of the user to delete")) - 1

    if user_index < 0 or user_index >= len(users):
        console.print("[bold red]❌ Invalid selection. Aborting.[/bold red]")
        return

    user_id, full_name, _, _ = users[user_index]

    # Confirm deletion
    if Confirm.ask(f"⚠️ Are you sure you want to delete [bold]{full_name}[/bold]?", default=False):
        cursor.execute("DELETE FROM users WHERE user_ID = %s", (user_id,))
        console.print(f"[green]✅ User [bold]{full_name}[/bold] has been deleted successfully.[/green]")
    else:
        console.print("[yellow]❎ Deletion cancelled.[/yellow]")

## 2.3 Display Users

In [ ]:
def display_users(cursor):
    console = Console()

    cursor.execute("""
        SELECT full_name, email, role, location, years_of_experience, preferred_learning_platform
        FROM users
        ORDER BY created_at DESC
    """)
    users = cursor.fetchall()

    if not users:
        console.print("[red bold]❌ No users found.[/red bold]")
        return

    user_table = Table(title="🧑‍💼 [bold blue]Registered Users[/bold blue]", box=box.DOUBLE)

    user_table.add_column("👤 Name", style="cyan", no_wrap=True)
    user_table.add_column("📧 Email", style="magenta")
    user_table.add_column("🎭 Role", style="green")
    user_table.add_column("📍 Location", style="yellow")
    user_table.add_column("📅 Exp", style="bright_blue", justify="center")
    user_table.add_column("📚 Platform", style="bold red")

    for full_name, email, role, location, exp, platform in users:
        user_table.add_row(
            full_name, email, role.title(), location or "🌍 N/A", str(exp), platform or "📘 N/A"
        )

    console.print(user_table)


In [ ]:
display_users(cursor)

                                          🧑‍💼 Registered Users                                           
╔══════════════════════╦════════════════════════╦════════════╦═════════════╦════════╦════════════════════╗
║ 👤 Name              ║ 📧 Email               ║ 🎭 Role    ║ 📍 Location ║ 📅 Exp ║ 📚 Platform        ║
╠══════════════════════╬════════════════════════╬════════════╬═════════════╬════════╬════════════════════╣
║ Test User for delete ║ Nan                    ║ Student    ║ Nan         ║   12   ║ Nan                ║
║ Mohamed Amr          ║ mohamed@gmail.com      ║ Admin      ║ asuuit      ║   12   ║ Nan                ║
║ Ahmed Mamdouh        ║ ahmed@gmail.com        ║ Instructor ║ Assuit      ║   12   ║ Nan                ║
║ Mohamed Hatem        ║ mohamedhatem@gmail.com ║ Instructor ║ cairo       ║   5    ║ Nan                ║
║ Omar Mohamed Nagib   ║ omarmohamed@gmail.com  ║ Instructor ║ Cairo       ║   5    ║ Udemy              ║
║ Basel Amr Barakat    ║ baselamr52@gmail.com   ║ Student    ║ Cairo       ║   2    ║ Udemy and Coursera ║
╚══════════════════════╩════════════════════════╩════════════╩═════════════╩════════╩════════════════════╝

In [ ]:
delete_user(cursor)

🗑️ Delete a User

                           👥 List of Users                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ No.    ┃ Full Name            ┃ Email                  ┃ Role       ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1      │ Mohamed Amr          │ mohamed@gmail.com      │ admin      │
├────────┼──────────────────────┼────────────────────────┼────────────┤
│ 2      │ Basel Amr Barakat    │ baselamr52@gmail.com   │ student    │
├────────┼──────────────────────┼────────────────────────┼────────────┤
│ 3      │ Test User for delete │ Nan                    │ student    │
├────────┼──────────────────────┼────────────────────────┼────────────┤
│ 4      │ Mohamed Hatem        │ mohamedhatem@gmail.com │ instructor │
├────────┼──────────────────────┼────────────────────────┼────────────┤
│ 5      │ Ahmed Mamdouh        │ ahmed@gmail.com        │ instructor │
├────────┼──────────────────────┼────────────────────────┼────────────┤
│ 6      │ Omar Mohamed Nagib   │ omarmohamed@gmail.com  │ instructor │
└────────┴──────────────────────┴────────────────────────┴────────────┘

Enter the number of the user to delete:

3


⚠️ Are you sure you want to delete Test User for delete? [y/n] (n):

y


✅ User Test User for delete has been deleted successfully.

In [ ]:
display_users(cursor)

                                         🧑‍💼 Registered Users                                          
╔════════════════════╦════════════════════════╦════════════╦═════════════╦════════╦════════════════════╗
║ 👤 Name            ║ 📧 Email               ║ 🎭 Role    ║ 📍 Location ║ 📅 Exp ║ 📚 Platform        ║
╠════════════════════╬════════════════════════╬════════════╬═════════════╬════════╬════════════════════╣
║ Mohamed Amr        ║ mohamed@gmail.com      ║ Admin      ║ asuuit      ║   12   ║ Nan                ║
║ Ahmed Mamdouh      ║ ahmed@gmail.com        ║ Instructor ║ Assuit      ║   12   ║ Nan                ║
║ Mohamed Hatem      ║ mohamedhatem@gmail.com ║ Instructor ║ cairo       ║   5    ║ Nan                ║
║ Omar Mohamed Nagib ║ omarmohamed@gmail.com  ║ Instructor ║ Cairo       ║   5    ║ Udemy              ║
║ Basel Amr Barakat  ║ baselamr52@gmail.com   ║ Student    ║ Cairo       ║   2    ║ Udemy and Coursera ║
╚════════════════════╩════════════════════════╩════════════╩═════════════╩════════╩════════════════════╝

# **3. Courses Table**

## **3.1 Add Course**

In [ ]:
import uuid

def get_instructors(cursor):
    """Fetch all instructors from the users table."""
    cursor.execute("SELECT user_ID, full_name FROM users WHERE role = 'instructor';")
    return cursor.fetchall()

def add_new_course_with_instructor(cursor, conn):
    # Fetch and display instructors
    instructors = get_instructors(cursor)
    if not instructors:
        print("⚠️ No instructors found in the database.")
        return

    print("\n📚 Available Instructors:")
    for idx, (instructor_id, full_name) in enumerate(instructors, 1):
        print(f"{idx}. {full_name} (ID: {instructor_id})")

    # Instructor selection with validation
    while True:
        try:
            instructor_choice = int(input("\nSelect an instructor by number: "))
            if 1 <= instructor_choice <= len(instructors):
                break
            else:
                print("❌ Invalid choice. Please enter a valid number.")
        except ValueError:
            print("❌ Please enter a numeric value.")

    selected_instructor_id = instructors[instructor_choice - 1][0]
    selected_instructor_name = instructors[instructor_choice - 1][1]

    print("\n📝 Enter Course Details:")
    course_id = str(uuid.uuid4())
    title = input("Course title: ").strip()
    description = input("Course description: ").strip()
    category = input("Course category: ").strip()

    # Course level selection with dropdown style menu
    levels = ['Beginner', 'Intermediate', 'Advanced']
    print("\n🎚️ Select course level:")
    for idx, lvl in enumerate(levels, 1):
        print(f"{idx}. {lvl}")
    while True:
        try:
            level_choice = int(input("Enter choice (1-3): "))
            if 1 <= level_choice <= 3:
                level = levels[level_choice - 1]
                break
            else:
                print("❌ Invalid level selection.")
        except ValueError:
            print("❌ Please enter a number from 1 to 3.")

    # Price input
    while True:
        try:
            price = float(input("Course price (e.g., 0.0 for free): "))
            if price >= 0:
                break
            else:
                print("❌ Price cannot be negative.")
        except ValueError:
            print("❌ Please enter a valid number.")

    thumbnail_url = input("Thumbnail URL (optional): ").strip()

    # Total hours
    while True:
        try:
            total_hours = float(input("Total course hours: "))
            if total_hours > 0:
                break
            else:
                print("❌ Hours must be greater than 0.")
        except ValueError:
            print("❌ Please enter a valid number.")

    what_you_will_learn = input("What will students learn? ").strip()
    who_this_course_is_for = input("Who is this course for? ").strip()

    # Insert query
    insert_query = '''
        INSERT INTO courses (
            course_ID, course_title, description, category, level, instructor_id,
            price, thumbnail_url, total_hours, created_at, updated_at,
            what_you_will_learn, who_this_course_is_for
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, %s, %s)
    '''
    course_data = (
        course_id, title, description, category, level, selected_instructor_id,
        price, thumbnail_url, total_hours, what_you_will_learn, who_this_course_is_for
    )

    try:
        cursor.execute(insert_query, course_data)
        conn.commit()
        print(f"\n✅ Course '{title}' added successfully with instructor '{selected_instructor_name}'!")
    except Exception as e:
        print(f"\n❌ Error while inserting course: {e}")


In [ ]:
#Add Course1
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 1

📝 Enter Course Details:
Course title: Machine Learning Specialization: Mastering the Art of Data Science
Course description:  Welcome to the Machine Learning Specialization! This comprehensive program is designed to take you from a beginner to an expert in the field of machine learning. Whether you're a data enthusiast, aspiring data scientist, or someone looking to expand your knowledge in AI and machine learning, this course will provide you with the necessary skills and hands-on experience to build impactful machine learning models.  Through a series of lessons and projects, you'll explore a wide range of machine learning techniques and algorithms, from supervised learning and regression models to unsupervised learning and deep learning. Yo

In [ ]:
#Add Course2
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 2

📝 Enter Course Details:
Course title: Introduction to python
Course description:  Welcome to the 100 Days of Code - The Complete Python Pro Bootcamp, the only course you need to learn to code with Python. With over 500,000 5 STAR reviews and a 4.8 average, my courses are some of the HIGHEST RATED courses in the history of Udemy!  
Course category: Programming Language

🎚️ Select course level:
1. Beginner
2. Intermediate
3. Advanced
Enter choice (1-3): 2
Course price (e.g., 0.0 for free): 0.0
Thumbnail URL (optional):  https://www.google.com/search?sca_esv=0c50508b885de737&sxsrf=AHTn8zqvp2gvnvkOUWaeQv84e_Uwpe0ZqA:1744405094404&q=python+course+png&udm=2&fbs=ABzOT_CWdhQLP1FcmU5B0fn3xuWpA-dk4wpBWOGsoR7DG5zJBpcx8kZB4NRoUjdgt8WwoMt2eLfGlVE3GaT9s2o0r

In [ ]:
#Add Course3
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 3

📝 Enter Course Details:
Course title: Introduction to Computer Vision and Image Processing
Course description: This course is designed to introduce learners to the fascinating world of computer vision and digital image processing. You'll start with the foundational concepts of how machines understand visual information, then dive deeper into the techniques used to process, filter, and extract features from images. With a mix of theory and practical knowledge, this course prepares you for advanced topics in AI, robotics, and visual recognition systems.  Throughout the course, you'll gain hands-on understanding of how images are represented digitally, how to enhance them through filtering, and how to extract meaningful information through edge d

In [ ]:
#Add Course4
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 2

📝 Enter Course Details:
Course title: OpenCV Python Tutorial For Beginners
Course description: This hands-on course introduces you to OpenCV, the most popular computer vision library, using the Python programming language. You'll start by learning how to read, display, and manipulate images, then progress to more advanced topics like drawing shapes, working with videos, and performing real-time image operations.  Through practical examples and beginner-friendly lessons, this course makes it easy to get started with OpenCV—even if you’ve never worked with images before.
Course category: OpenCV

🎚️ Select course level:
1. Beginner
2. Intermediate
3. Advanced
Enter choice (1-3): 3
Course price (e.g., 0.0 for free): 0.0
Thumbnail URL (optional): h

In [ ]:
#Add Course5
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 1

📝 Enter Course Details:
Course title: Mastering Natural Language Processing (NLP) with Python
Course description: This comprehensive course is designed to take learners from the foundational concepts of Natural Language Processing to advanced applications using Python. Through hands-on lessons and practical examples, you will learn how machines understand, interpret, and generate human language. Whether you're looking to build chatbots, perform sentiment analysis, or dive into language modeling, this course equips you with the skills needed to succeed in the world of NLP.
Course category: NLP

🎚️ Select course level:
1. Beginner
2. Intermediate
3. Advanced
Enter choice (1-3): 3
Course price (e.g., 0.0 for free): 0.0
Thumbnail URL (optional): h

In [ ]:
#Add Course5 to test delete
add_new_course_with_instructor(cursor, db_conn)


📚 Available Instructors:
1. Mohamed Hatem (ID: b002340e-eccf-4084-893f-24ef4a40df21)
2. Ahmed Mamdouh (ID: b817a724-39a5-495a-baec-80012db6069f)
3. Omar Mohamed Nagib (ID: bd537750-817e-4c53-86ea-0576f7a21b4b)

Select an instructor by number: 2

📝 Enter Course Details:
Course title: Temp Course to test delete
Course description: No Description
Course category: No Category

🎚️ Select course level:
1. Beginner
2. Intermediate
3. Advanced
Enter choice (1-3): 1
Course price (e.g., 0.0 for free): 12
Thumbnail URL (optional): No URL
Total course hours: 12
What will students learn? Nothing
Who is this course for? Noone

✅ Course 'Temp Course to test delete' added successfully with instructor 'Ahmed Mamdouh'!


## **3.2 Update The Course**

In [ ]:
def get_courses(cursor):
    """Fetch all courses for selection."""
    cursor.execute("SELECT course_ID, course_title FROM courses")
    return cursor.fetchall()

def update_course(cursor, conn):
    courses = get_courses(cursor)
    if not courses:
        print("⚠️ No courses found in the database.")
        return

    print("\n🎓 Available Courses:")
    for idx, (course_id, course_title) in enumerate(courses, 1):
        print(f"{idx}. {course_title} (ID: {course_id})")

    while True:
        try:
            course_choice = int(input("\nSelect a course to update by number: "))
            if 1 <= course_choice <= len(courses):
                break
            else:
                print("❌ Invalid choice. Try again.")
        except ValueError:
            print("❌ Enter a number.")

    course_id = courses[course_choice - 1][0]

    print("\n🛠️ What would you like to update?")
    options = [
        "Course Title", "Description", "Category", "Level",
        "Instructor", "Price", "Thumbnail URL", "Total Hours",
        "What You Will Learn", "Who This Course Is For"
    ]
    for idx, opt in enumerate(options, 1):
        print(f"{idx}. {opt}")

    choice = int(input("\nSelect an option by number: "))
    if choice < 1 or choice > len(options):
        print("❌ Invalid option.")
        return

    column_map = {
        1: "course_title",
        2: "description",
        3: "category",
        4: "level",
        5: "instructor_id",
        6: "price",
        7: "thumbnail_url",
        8: "total_hours",
        9: "what_you_will_learn",
        10: "who_this_course_is_for"
    }

    column = column_map[choice]

    if column == "instructor_id":
        instructors = get_instructors(cursor)
        for idx, (_, name) in enumerate(instructors, 1):
            print(f"{idx}. {name}")
        idx = int(input("Choose new instructor: "))
        new_value = instructors[idx - 1][0]
    elif column in ["price", "total_hours"]:
        new_value = float(input(f"Enter new {options[choice - 1]}: "))
    else:
        new_value = input(f"Enter new {options[choice - 1]}: ")

    try:
        query = f"UPDATE courses SET {column} = %s, updated_at = CURRENT_TIMESTAMP WHERE course_ID = %s"
        cursor.execute(query, (new_value, course_id))
        conn.commit()
        print("✅ Course updated successfully!")
    except Exception as e:
        print(f"❌ Failed to update course: {e}")


In [ ]:
update_course(cursor, db_conn)


🎓 Available Courses:
1. Temp Course to test delete (ID: 0cfe63bc-0c53-4b5b-ac20-965b8eeef9a1)
2. Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)
3. Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)
4. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
5. OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)
6. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)

Select a course to update by number: 1

🛠️ What would you like to update?
1. Course Title
2. Description
3. Category
4. Level
5. Instructor
6. Price
7. Thumbnail URL
8. Total Hours
9. What You Will Learn
10. Who This Course Is For

Select an option by number: 1
Enter new Course Title: The Name is updated
✅ Course updated successfully!


## **3.3 Delete Course**

In [ ]:
def delete_course(cursor, conn):
    courses = get_courses(cursor)
    if not courses:
        console.print("[bold yellow]⚠️ No courses available.[/bold yellow]")
        return

    table = Table(title="📕 Available Courses")
    table.add_column("No.", style="cyan", justify="right")
    table.add_column("Course Title", style="magenta")
    table.add_column("Course ID", style="dim")

    for idx, (course_id, course_title) in enumerate(courses, 1):
        table.add_row(str(idx), course_title, course_id)

    console.print(table)

    while True:
        choice = input("\n[Press 'q' to quit] Select a course to delete by number: ").strip().lower()
        if choice == 'q':
            console.print("[bold blue]🚪 Exiting course deletion...[/bold blue]")
            return
        if choice.isdigit():
            choice = int(choice)
            if 1 <= choice <= len(courses):
                break
            else:
                console.print("[red]❌ Invalid number. Try again.[/red]")
        else:
            console.print("[red]❌ Please enter a number or 'q' to quit.[/red]")

    course_id, course_title = courses[choice - 1]

    confirm = input(f"⚠️ Are you sure you want to delete [bold]{course_title}[/bold]? (yes/no): ").strip().lower()
    if confirm != "yes":
        console.print("[blue]❎ Deletion cancelled.[/blue]")
        return

    try:
        cursor.execute("DELETE FROM courses WHERE course_ID = %s", (course_id,))
        conn.commit()
        console.print(f"[bold green]✅ Course '{course_title}' deleted successfully.[/bold green]")
    except Exception as e:
        console.print(f"[bold red]❌ Error deleting course: {e}[/bold red]")

In [ ]:
delete_course(cursor,db_conn)


📕 Courses:
1. The Name is updated (ID: 0cfe63bc-0c53-4b5b-ac20-965b8eeef9a1)
2. Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)
3. Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)
4. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
5. OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)
6. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)

Select a course to delete by number: 1
⚠️ Are you sure you want to delete 'The Name is updated'? (yes/no): yes
✅ Course 'The Name is updated' deleted successfully.


In [ ]:
# Check if the course is deleted
delete_course(cursor,db_conn)

                                               📕 Available Courses                                                
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Course Title                                                       ┃ Course ID                            ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Introduction to Computer Vision and Image Processing               │ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │
│   2 │ Mastering Natural Language Processing (NLP) with Python            │ 4c53688f-430a-4d8c-b21c-e796256b46da │
│   3 │ Machine Learning Specialization: Mastering the Art of Data Science │ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │
│   4 │ OpenCV Python Tutorial For Beginners                               │ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │
│   5 │ Introduction to python                                             │ fdd32b6c-cad7-4546-b958-41bf41429b3f │
└─────┴────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘


[Press 'q' to quit] Select a course to delete by number: q


🚪 Exiting course deletion...

## **3.4 Display the Courses**

In [ ]:
def display_all_courses(cursor):
    """
    Display all courses with their full details including instructor info.
    """
    query = '''
        SELECT
            c.course_ID,
            c.course_title,
            c.description,
            c.category,
            c.level,
            c.price,
            c.thumbnail_url,
            c.total_hours,
            c.created_at,
            c.updated_at,
            c.what_you_will_learn,
            c.who_this_course_is_for,
            u.full_name AS instructor_name
        FROM courses c
        JOIN users u ON c.instructor_id = u.user_ID
        ORDER BY c.created_at DESC;
    '''
    cursor.execute(query)
    courses = cursor.fetchall()

    if not courses:
        print("📭 No courses found.")
        return

    print("\n🎓 All Courses:\n" + "-" * 70)
    for idx, course in enumerate(courses, 1):
        (
            course_id, title, desc, category, level, price, thumb, hours,
            created_at, updated_at, learn, target, instructor
        ) = course

        print(f"\n📘 Course {idx}: {title}")
        print(f"🆔 ID: {course_id}")
        print(f"👨‍🏫 Instructor: {instructor}")
        print(f"📚 Category: {category}")
        print(f"🎯 Level: {level}")
        print(f"💰 Price: ${price:.2f}")
        print(f"⏱ Total Hours: {hours} hours")
        print(f"🗓 Created At: {created_at}")
        print(f"🔄 Updated At: {updated_at}")
        print(f"🌐 Thumbnail URL: {thumb if thumb else 'N/A'}")
        print(f"📖 Description: {desc}")
        print(f"✅ What You'll Learn: {learn}")
        print(f"👥 Target Audience: {target}")
        print("-" * 70)


In [ ]:
display_all_courses(cursor)


🎓 All Courses:
----------------------------------------------------------------------

📘 Course 1: Mastering Natural Language Processing (NLP) with Python
🆔 ID: 4c53688f-430a-4d8c-b21c-e796256b46da
👨‍🏫 Instructor: Mohamed Hatem
📚 Category: NLP
🎯 Level: Advanced
💰 Price: $0.00
⏱ Total Hours: 5.0 hours
🗓 Created At: 2025-04-13 01:10:16
🔄 Updated At: 2025-04-13 01:10:16
🌐 Thumbnail URL: https://www.google.com/search?sca_esv=b4544fb49b72c327&sxsrf=AHTn8zp3Xv_zJmqKJWI3FU0QW1fLxwtFyA:1744506582888&q=NLP+Course+png&udm=2&fbs=ABzOT_CWdhQLP1FcmU5B0fn3xuWpA-dk4wpBWOGsoR7DG5zJBpcx8kZB4NRoUjdgt8WwoMt2eLfGlVE3GaT9s2o0rEuuXaa3Xp90ImSjm6VOFeSjw-sbWSvWYDydvJUeUSyzm0blZO8Fo08bVHHpJJMqbH43eiwpBGOUaDdYnv1Z5jLaIOss4Dvt_VUtx_IO54Nb8rDVUhwRcvJSBS--bSqCYP2LCffS5g&sa=X&ved=2ahUKEwjHxLm56dOMAxVthf0HHQFTHN4QtKgLegQIERAB&biw=1536&bih=730&dpr=1.25#vhid=A2jqH0olMtyQjM&vssid=mosaic
📖 Description: This comprehensive course is designed to take learners from the foundational concepts of Natural Language Processing to

# **4. Add Course Chapter and content**

## **4.1 Add Course Content**

In [ ]:
# 1️⃣ Show available courses
def choose_course(cursor):
    cursor.execute("SELECT course_ID, course_title FROM courses")
    courses = cursor.fetchall()
    if not courses:
        console.print("⚠️ [bold red]No courses found.[/bold red]", style="bold red")
        return None

    table = Table(show_header=True, header_style="bold magenta", box=box.SIMPLE)
    table.add_column("Course ID", style="dim")
    table.add_column("Course Title")

    for idx, (cid, title) in enumerate(courses, 1):
        table.add_row(str(cid), title)

    console.print(table)

    while True:
        try:
            choice = int(input(f"{Fore.GREEN}➡️ {Fore.WHITE}Choose a course by number: "))
            if 1 <= choice <= len(courses):
                return courses[choice - 1][0]
            else:
                console.print(f"{Fore.RED}❌ Invalid number. Please select a valid course number.", style="bold red")
        except ValueError:
            console.print(f"{Fore.RED}❌ Please enter a valid number.", style="bold red")

In [ ]:
# 2️⃣ Delete all chapters for a course
def delete_chapters_for_course(cursor, conn, course_id):
    console.print(f"🧹 [bold yellow]Deleting all chapters for course ID:[/bold yellow] {course_id}")

    cursor.execute("SELECT chapter_ID FROM course_chapters WHERE course_ID = %s", (course_id,))
    chapter_ids = [row[0] for row in cursor.fetchall()]

    if not chapter_ids:
        console.print("ℹ️ No chapters to delete.", style="bold green")
        return

    for chapter_id in chapter_ids:
        cursor.execute("DELETE FROM chapter_content WHERE chapter_ID = %s", (chapter_id,))
        cursor.execute("DELETE FROM course_chapters WHERE course_ID = %s AND chapter_ID = %s", (course_id, chapter_id))
        cursor.execute("DELETE FROM chapters WHERE chapter_ID = %s", (chapter_id,))
        console.print(f"✅ Deleted chapter ID: {chapter_id}", style="bold red")

    conn.commit()
    console.print("🧼 [bold green]All chapters and links removed.[/bold green]")


In [ ]:
# 3️⃣ Helper: Get chapter position
def get_next_chapter_position(cursor, course_id):
    cursor.execute("SELECT COUNT(*) FROM course_chapters WHERE course_ID = %s", (course_id,))
    return cursor.fetchone()[0] + 1

In [ ]:
# 4️⃣ Add new chapter and link
def add_chapter(cursor, conn, course_id):
    title = input("📘 Enter chapter title: ")
    description = input("📝 Enter chapter description: ")
    cursor.execute("INSERT INTO chapters (title, description) VALUES (%s, %s)", (title, description))
    conn.commit()
    chapter_id = cursor.lastrowid
    console.print(f"✅ [bold green]Chapter '{title}' created successfully with ID: {chapter_id}.[/bold green]")

    position = get_next_chapter_position(cursor, course_id)
    cursor.execute("INSERT INTO course_chapters (course_ID, chapter_ID, position) VALUES (%s, %s, %s)",
                   (course_id, chapter_id, position))
    conn.commit()
    console.print(f"🔗 [bold blue]Linked chapter to course at position {position}.[/bold blue]")

    return chapter_id

In [ ]:
# 5️⃣ Helper: Get content position in chapter
def get_next_content_position(cursor, chapter_id):
    # Get the next available position for the content in the chapter
    cursor.execute("SELECT MAX(position) FROM chapter_content WHERE chapter_ID = %s", (chapter_id,))
    result = cursor.fetchone()
    return (result[0] + 1) if result[0] else 1

In [ ]:
# 6️⃣ Add content to a chapter (automatically create content if not found)
def add_content_to_chapter(cursor, conn, chapter_id):
    while True:
        try:
            # Step 1: Ask for content type
            content_type = input("📦 Content type (video/pdf/quiz/ppt/text/file): ").lower()
            description = input("📝 Description: ")
            duration = float(input("⏱ Duration (minutes): "))

            # Step 2: Ask for URL after getting content type, description, and duration
            content_url = input("🔗 Content URL (Please provide a unique URL): ")

            # Step 3: Check if content already exists with the same URL
            cursor.execute("SELECT content_ID FROM content WHERE content_url = %s", (content_url,))
            existing_content = cursor.fetchone()

            if existing_content:
                content_id = existing_content[0]
                # Step 4: Delete all existing references of the content in chapter_content
                cursor.execute("DELETE FROM chapter_content WHERE content_ID = %s", (content_id,))
                conn.commit()
                console.print(f"✅ Existing content with URL '{content_url}' replaced.")

                # Step 5: Delete the old content from the content table
                cursor.execute("DELETE FROM content WHERE content_ID = %s", (content_id,))
                conn.commit()

            # Step 6: Insert the new content into the content table
            cursor.execute("""
                INSERT INTO content (content_type, content_url, description, duration)
                VALUES (%s, %s, %s, %s)
            """, (content_type, content_url, description, duration))
            conn.commit()
            content_id = cursor.lastrowid
            console.print(f"✅ New content created successfully at ID: {content_id}")

            # Step 7: Link content to the chapter
            position = get_next_content_position(cursor, chapter_id)
            cursor.execute("""
                INSERT INTO chapter_content (chapter_ID, content_ID, position)
                VALUES (%s, %s, %s)
            """, (chapter_id, content_id, position))
            conn.commit()
            console.print(f"✅ Content linked to chapter ID {chapter_id} at position {position}")

            # Step 8: Ask if the user wants to add more content
            if input(f"➕ Add another content to this chapter? (y/n): ").lower() != 'y':
                break

        except IntegrityError as e:
            console.print(f"🚫 Integrity error occurred: {e}", style="bold red")
            conn.rollback()  # Rollback in case of error
            break  # Exit loop on error
        except ValueError:
            console.print("❌ Invalid input. Please enter valid data.", style="bold red")
            continue  # Skip and ask for the correct input again

In [ ]:
# Main function to run the course chapter setup tool
def run():
    console.print("[bold blue]===== COURSE CHAPTER SETUP TOOL =====[/bold blue]")

    # Step 1: Choose a course
    course_id = choose_course(cursor)

    if course_id:
        # Step 2: Prompt for clearing existing chapters
        clear = input(f"{Fore.CYAN}⚠️ Do you want to delete all existing chapters for this course? (y/n): {Fore.WHITE}").lower()

        # Step 3: Delete existing chapters if the user chooses 'y'
        if clear == 'y':
            delete_chapters_for_course(cursor, db_conn, course_id)

        # Step 4: Adding new chapters to the course
        while True:
            # Step 4.1: Add a chapter and get the chapter ID
            chapter_id = add_chapter(cursor, db_conn, course_id)

            # Step 4.2: Add content to the chapter
            success = add_content_to_chapter(cursor, db_conn, chapter_id)

            # Step 4.3: Check if content was successfully added
            if not success:
                console.print(f"{Fore.RED}❗ Please re-enter the chapter with a new content URL.", style="bold red")

            # Step 4.4: Prompt the user if they want to add another chapter
            again = input(f"{Fore.YELLOW}➕ Add another chapter? (y/n): {Fore.WHITE}").lower()
            if again != 'y':
                break

### **4.1.1 Add Course 1**

In [ ]:
if __name__ == "__main__":
    run()

===== COURSE CHAPTER SETUP TOOL =====

 Course ID                              Course Title                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  537ad633-a6f1-4e39-bb62-0f5b80726f21   Machine Learning Specialization: Mastering the Art of Data Science  
  fdd32b6c-cad7-4546-b958-41bf41429b3f   Introduction to python

➡️ Choose a course by number: 1
⚠️ Do you want to delete all existing chapters for this course? (y/n): y


🧹 Deleting all chapters for course ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21

ℹ️ No chapters to delete.

...

✅ New content created successfully at ID: 18

✅ Content linked to chapter ID 3 at position 6

➕ Add another content to this chapter? (y/n): n


[31m❗ Please re-enter the chapter with a new content URL.

➕ Add another chapter? (y/n): n


### **4.1.2 Add Course 2**

In [ ]:
if __name__ == "__main__":
    run()

===== COURSE CHAPTER SETUP TOOL =====

 Course ID                              Course Title                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  537ad633-a6f1-4e39-bb62-0f5b80726f21   Machine Learning Specialization: Mastering the Art of Data Science  
  fdd32b6c-cad7-4546-b958-41bf41429b3f   Introduction to python

➡️ Choose a course by number: 2
⚠️ Do you want to delete all existing chapters for this course? (y/n): y


🧹 Deleting all chapters for course ID: fdd32b6c-cad7-4546-b958-41bf41429b3f

ℹ️ No chapters to delete.

...

✅ New content created successfully at ID: 28

✅ Content linked to chapter ID 5 at position 4

➕ Add another content to this chapter? (y/n): n


[31m❗ Please re-enter the chapter with a new content URL.

➕ Add another chapter? (y/n): n


### **4.1.3 Add Course 3**

In [ ]:
if __name__ == "__main__":
    run()

===== COURSE CHAPTER SETUP TOOL =====

 Course ID                              Course Title                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a   Introduction to Computer Vision and Image Processing                
  4c53688f-430a-4d8c-b21c-e796256b46da   Mastering Natural Language Processing (NLP) with Python             
  537ad633-a6f1-4e39-bb62-0f5b80726f21   Machine Learning Specialization: Mastering the Art of Data Science  
  a2ab7375-b89a-48c8-b56e-ce7152b6ec96   OpenCV Python Tutorial For Beginners                                
  fdd32b6c-cad7-4546-b958-41bf41429b3f   Introduction to python

➡️ Choose a course by number: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a


[31m❌ Please enter a valid number.

➡️ Choose a course by number: 1
⚠️ Do you want to delete all existing chapters for this course? (y/n): y


...

✅ New content created successfully at ID: 36

✅ Content linked to chapter ID 7 at position 4

➕ Add another content to this chapter? (y/n): n


[31m❗ Please re-enter the chapter with a new content URL.

➕ Add another chapter? (y/n): n


### **4.1.4 Add Course 4**

In [ ]:
if __name__ == "__main__":
    run()

===== COURSE CHAPTER SETUP TOOL =====

 Course ID                              Course Title                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a   Introduction to Computer Vision and Image Processing                
  4c53688f-430a-4d8c-b21c-e796256b46da   Mastering Natural Language Processing (NLP) with Python             
  537ad633-a6f1-4e39-bb62-0f5b80726f21   Machine Learning Specialization: Mastering the Art of Data Science  
  a2ab7375-b89a-48c8-b56e-ce7152b6ec96   OpenCV Python Tutorial For Beginners                                
  fdd32b6c-cad7-4546-b958-41bf41429b3f   Introduction to python

➡️ Choose a course by number: 4
⚠️ Do you want to delete all existing chapters for this course? (y/n): y


🧹 Deleting all chapters for course ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96

ℹ️ No chapters to delete.

...

✅ New content created successfully at ID: 48

✅ Content linked to chapter ID 9 at position 6

➕ Add another content to this chapter? (y/n): n


[31m❗ Please re-enter the chapter with a new content URL.

➕ Add another chapter? (y/n): n


### **4.1.5 Add Course 5**

In [ ]:
if __name__ == "__main__":
    run()

===== COURSE CHAPTER SETUP TOOL =====

 Course ID                              Course Title                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a   Introduction to Computer Vision and Image Processing                
  4c53688f-430a-4d8c-b21c-e796256b46da   Mastering Natural Language Processing (NLP) with Python             
  537ad633-a6f1-4e39-bb62-0f5b80726f21   Machine Learning Specialization: Mastering the Art of Data Science  
  a2ab7375-b89a-48c8-b56e-ce7152b6ec96   OpenCV Python Tutorial For Beginners                                
  fdd32b6c-cad7-4546-b958-41bf41429b3f   Introduction to python

➡️ Choose a course by number: 2
⚠️ Do you want to delete all existing chapters for this course? (y/n): y


🧹 Deleting all chapters for course ID: 4c53688f-430a-4d8c-b21c-e796256b46da

ℹ️ No chapters to delete.

...

✅ New content created successfully at ID: 66

✅ Content linked to chapter ID 12 at position 6

➕ Add another content to this chapter? (y/n): n


[31m❗ Please re-enter the chapter with a new content URL.

➕ Add another chapter? (y/n): n


## **4.2 Update Content Title**

In [ ]:
from rich.console import Console
from rich.table import Table
from rich.box import ROUNDED  # Using ROUNDED as the box style

def get_courses(cursor):
    # Fetch all courses from the database
    cursor.execute("SELECT course_ID, course_title FROM courses")
    return cursor.fetchall()

def get_chapters_for_course(cursor, course_id):
    # Fetch all chapters for the selected course
    cursor.execute("""
        SELECT chapter_ID, title FROM chapters
        WHERE chapter_ID IN (
            SELECT chapter_ID FROM course_chapters WHERE course_ID = %s
        )
    """, (course_id,))
    return cursor.fetchall()

def get_content_for_chapter(cursor, chapter_id):
    # Fetch all content for the selected chapter
    cursor.execute("""
        SELECT content_ID, content_type, content_url, content_title FROM content
        WHERE content_ID IN (
            SELECT content_ID FROM chapter_content WHERE chapter_ID = %s
        )
    """, (chapter_id,))
    return cursor.fetchall()

def update_content_title(conn, cursor, content_id, new_title):
    # Update the content title for the selected content ID
    cursor.execute("""
        UPDATE content
        SET content_title = %s
        WHERE content_ID = %s
    """, (new_title, content_id))
    conn.commit()  # Commit the transaction using the connection

def display_courses_and_update_titles(conn, cursor):
    console = Console()

    # Step 1: Fetch all courses
    courses = get_courses(cursor)
    if not courses:
        console.print("⚠️ [bold red]No courses found.[/bold red]")
        return

    # Step 2: Display courses and let the user select one
    course_table = Table(title="📚 Available Courses", box=ROUNDED)
    course_table.add_column("ID", justify="center")
    course_table.add_column("Title", justify="left")

    for course_id, course_title in courses:
        course_table.add_row(str(course_id), course_title)

    console.print(course_table)

    # Step 3: Prompt the user to select a course
    selected_course_id = input("\nEnter the Course ID you want to update: ")

    # Step 4: Fetch and display the chapters for the selected course
    chapters = get_chapters_for_course(cursor, selected_course_id)
    if not chapters:
        console.print("⚠️ [bold red]No chapters found for this course.[/bold red]")
        return

    chapter_table = Table(title="📚 Chapters for Selected Course", box=ROUNDED)
    chapter_table.add_column("ID", justify="center")
    chapter_table.add_column("Title", justify="left")

    for chapter_id, chapter_title in chapters:
        chapter_table.add_row(str(chapter_id), chapter_title)

    console.print(chapter_table)

    # Step 5: For each chapter, display content and update titles
    for chapter_id, chapter_title in chapters:
        console.print(f"\n🔖 [bold blue]Chapter: {chapter_title}[/bold blue]")

        content_items = get_content_for_chapter(cursor, chapter_id)
        if not content_items:
            console.print(f"   ⚠️ [bold red]No content found for this chapter.[/bold red]")
            continue

        content_table = Table(title=f"📦 Content for Chapter: {chapter_title}", box=ROUNDED)
        content_table.add_column("ID", justify="center")
        content_table.add_column("Type", justify="center")
        content_table.add_column("URL", justify="center")
        content_table.add_column("Title", justify="left")

        for content_id, content_type, content_url, content_title in content_items:
            content_table.add_row(str(content_id), content_type, content_url, content_title)

        console.print(content_table)

        # Step 6: Let the user update the title for each content item
        for content_id, _, _, current_title in content_items:
            new_title = input(f"\nEnter new title for content ID {content_id} (current title: {current_title}): ")
            if new_title.strip():  # Only update if new title is provided
                update_content_title(conn, cursor, content_id, new_title)
                console.print(f"[bold green]Content title updated successfully for content ID {content_id}.[/bold green]")
            else:
                console.print(f"[bold yellow]No update made for content ID {content_id}.[/bold yellow]")

# Example usage:
# Assuming you have a valid connection object 'conn' and cursor 'cursor'
# display_courses_and_update_titles(conn, cursor)


### **4.2.1 Update Content of course 1**

In [ ]:
display_courses_and_update_titles(db_conn, cursor)

                                            📚 Available Courses                                             
╭──────────────────────────────────────┬────────────────────────────────────────────────────────────────────╮
│                  ID                  │ Title                                                              │
├──────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │ Machine Learning Specialization: Mastering the Art of Data Science │
│ fdd32b6c-cad7-4546-b958-41bf41429b3f │ Introduction to python                                             │
╰──────────────────────────────────────┴────────────────────────────────────────────────────────────────────╯


Enter the Course ID you want to update: 537ad633-a6f1-4e39-bb62-0f5b80726f21


          📚 Chapters for Selected Course           
╭────┬─────────────────────────────────────────────╮
│ ID │ Title                                       │
├────┼─────────────────────────────────────────────┤
│ 1  │ Chapter 1: Introduction to Machine Learning │
│ 2  │ Chapter 2: What is Machine Learning?        │
│ 3  │ Chapter 3: Supervised Learning – Part 1     │
╰────┴─────────────────────────────────────────────╯

🔖 Chapter: Chapter 1: Introduction to Machine Learning

                        📦 Content for Chapter: Chapter 1: Introduction to Machine Learning                        
╭────┬───────┬────────────────────────────────────────────────────────┬───────────────────────────────────────────╮
│ ID │ Type  │                          URL                           │ Title                                     │
├────┼───────┼────────────────────────────────────────────────────────┼───────────────────────────────────────────┤
│ 1  │ video │ https://www.youtube.com/watch?v=wiNXzydta4c&list=PLkD… │ Lesson 1: Application of Machine Learning │
│ 2  │  pdf  │                       temp_url_1                       │ Lesson 2: What is Machine Learning        │
│ 3  │ video │ https://www.youtube.com/watch?v=XtlwSmJfUs4&list=PLkD… │ Lesson 3: Supervised Learning – Part 1    │
│ 4  │  pdf  │                       temp_url_2                       │ Lesson 4: Supervised Learning Part 2      │
│ 5  │ video │ https://www.youtube.com/watch?v=sca5rQ9x1cA&list=PLkD… │ Lesson 5: Unsupervised Learning – Part 1  │
│ 6  │  pdf  │                       temp_url_3                       │ Lesson 6: Unsupervised Learning – Part 2  │
╰────┴───────┴────────────────────────────────────────────────────────┴───────────────────────────────────────────╯

...

Content title updated successfully for content ID 16.


Enter new title for content ID 17 (current title: temp): Lesson 9: The Cost Function video


Content title updated successfully for content ID 17.


Enter new title for content ID 18 (current title: temp): Lesson 9: The Cost Function pdf


Content title updated successfully for content ID 18.

### **4.2.2 Update Content of course 2**

In [ ]:
display_courses_and_update_titles(db_conn, cursor)

                                            📚 Available Courses                                             
╭──────────────────────────────────────┬────────────────────────────────────────────────────────────────────╮
│                  ID                  │ Title                                                              │
├──────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │ Machine Learning Specialization: Mastering the Art of Data Science │
│ fdd32b6c-cad7-4546-b958-41bf41429b3f │ Introduction to python                                             │
╰──────────────────────────────────────┴────────────────────────────────────────────────────────────────────╯


Enter the Course ID you want to update: fdd32b6c-cad7-4546-b958-41bf41429b3f


           📚 Chapters for Selected Course            
╭────┬───────────────────────────────────────────────╮
│ ID │ Title                                         │
├────┼───────────────────────────────────────────────┤
│ 4  │ Chapter 1  : Working with variables in python │
│ 5  │ Chapter 2 : Control flow and Operators        │
╰────┴───────────────────────────────────────────────╯

🔖 Chapter: Chapter 1  : Working with variables in python

                       📦 Content for Chapter: Chapter 1  : Working with variables in python                       
╭────┬───────┬─────────────────────────────────────────────────────────────────────────────────────────┬──────────╮
│ ID │ Type  │                                           URL                                           │ Title    │
├────┼───────┼─────────────────────────────────────────────────────────────────────────────────────────┼──────────┤
│ 19 │ video │   https://www.youtube.com/watch?v=mvZHDpCHphk&list=PLDoPjvoNmBAyE_gei5d18qkfIe-Z8mocs   │ Untitled │
│ 20 │  pdf  │                                       temp_url_21                                       │ Untitled │
│ 21 │ video │ https://www.youtube.com/watch?v=xiMHoMVWdI4&list=PLDoPjvoNmBAyE_gei5d18qkfIe-Z8mocs&in… │ Untitled │
│ 22 │  pdf  │                                       temp_url_22                                       │ Untitled │
│ 23 │ video │ https://www.youtube.com/watch?v=YsENRLNaYug&list=PLDoPjvoNmBAyE_gei5d18qkfIe-Z8mocs&in… │ Untitled │
│ 24 │  pdf  │                                       temp_url_23                                       │ Untitled │
╰────┴───────┴─────────────────────────────────────────────────────────────────────────────────────────┴──────────╯

...

Content title updated successfully for content ID 26.


Enter new title for content ID 27 (current title: Untitled): Lesson 5: Functions video


Content title updated successfully for content ID 27.


Enter new title for content ID 28 (current title: Untitled): Lesson 5: Functions pdf


Content title updated successfully for content ID 28.

### **4.2.3 Update Content of course 3**

In [ ]:
display_courses_and_update_titles(db_conn, cursor)

                                            📚 Available Courses                                             
╭──────────────────────────────────────┬────────────────────────────────────────────────────────────────────╮
│                  ID                  │ Title                                                              │
├──────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │ Introduction to Computer Vision and Image Processing               │
│ 4c53688f-430a-4d8c-b21c-e796256b46da │ Mastering Natural Language Processing (NLP) with Python            │
│ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │ Machine Learning Specialization: Mastering the Art of Data Science │
│ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │ OpenCV Python Tutorial For Beginners                               │
│ fdd32b6c-cad7-4546-b958-41bf41429b3f │ Introduction to python                                             │
╰──────────────────────────────────────┴────────────────────────────────────────────────────────────────────╯


Enter the Course ID you want to update: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a


                    📚 Chapters for Selected Course                    
╭────┬────────────────────────────────────────────────────────────────╮
│ ID │ Title                                                          │
├────┼────────────────────────────────────────────────────────────────┤
│ 6  │ Chapter 1: Foundations of Computer Vision and Image Processing │
│ 7  │ Chapter 2: Image Filtering and Edge Detection                  │
╰────┴────────────────────────────────────────────────────────────────╯

🔖 Chapter: Chapter 1: Foundations of Computer Vision and Image Processing

              📦 Content for Chapter: Chapter 1: Foundations of Computer Vision and Image Processing               
╭────┬───────┬─────────────────────────────────────────────────────────────────────────────────────────┬──────────╮
│ ID │ Type  │                                           URL                                           │ Title    │
├────┼───────┼─────────────────────────────────────────────────────────────────────────────────────────┼──────────┤
│ 29 │ video │ https://www.youtube.com/watch?v=yetsJWgYh0o&list=PLwdnzlV3ogoVsma5GmBSsgJM6gHv1QoAo&in… │ Untitled │
│ 30 │  pdf  │                                        no_url_1                                         │ Untitled │
│ 31 │ video │ https://www.youtube.com/watch?v=1I6kfkY4GyQ&list=PLwdnzlV3ogoVsma5GmBSsgJM6gHv1QoAo&in… │ Untitled │
│ 32 │  pdf  │                                        no_url_2                                         │ Untitled │
╰────┴───────┴─────────────────────────────────────────────────────────────────────────────────────────┴──────────╯

...

Content title updated successfully for content ID 34.


Enter new title for content ID 35 (current title: Untitled): Lesson 4: Edge Detection and Feature Extraction


Content title updated successfully for content ID 35.


Enter new title for content ID 36 (current title: Untitled): Lesson 4: Edge Detection and Feature Extraction pdf


Content title updated successfully for content ID 36.

### **4.2.4 Update Content of course 4**

In [ ]:
display_courses_and_update_titles(db_conn, cursor)

                                            📚 Available Courses                                             
╭──────────────────────────────────────┬────────────────────────────────────────────────────────────────────╮
│                  ID                  │ Title                                                              │
├──────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │ Introduction to Computer Vision and Image Processing               │
│ 4c53688f-430a-4d8c-b21c-e796256b46da │ Mastering Natural Language Processing (NLP) with Python            │
│ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │ Machine Learning Specialization: Mastering the Art of Data Science │
│ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │ OpenCV Python Tutorial For Beginners                               │
│ fdd32b6c-cad7-4546-b958-41bf41429b3f │ Introduction to python                                             │
╰──────────────────────────────────────┴────────────────────────────────────────────────────────────────────╯


Enter the Course ID you want to update: a2ab7375-b89a-48c8-b56e-ce7152b6ec96


              📚 Chapters for Selected Course               
╭────┬─────────────────────────────────────────────────────╮
│ ID │ Title                                               │
├────┼─────────────────────────────────────────────────────┤
│ 8  │ Chapter 1: Getting Started with OpenCV              │
│ 9  │ Chapter 2: Working with Videos and Image Operations │
╰────┴─────────────────────────────────────────────────────╯

🔖 Chapter: Chapter 1: Getting Started with OpenCV

                          📦 Content for Chapter: Chapter 1: Getting Started with OpenCV                           
╭────┬───────┬─────────────────────────────────────────────────────────────────────────────────────────┬──────────╮
│ ID │ Type  │                                           URL                                           │ Title    │
├────┼───────┼─────────────────────────────────────────────────────────────────────────────────────────┼──────────┤
│ 37 │ video │   https://www.youtube.com/watch?v=kdLM6AOd2vc&list=PLS1QulWo1RIa7D1O6skqDQ-JZ1GGHKK-K   │ Untitled │
│ 38 │  pdf  │                                        no_url_41                                        │ Untitled │
│ 39 │ video │ https://www.youtube.com/watch?v=d3AT9EGp4iw&list=PLS1QulWo1RIa7D1O6skqDQ-JZ1GGHKK-K&in… │ Untitled │
│ 40 │  pdf  │                                        no_url_42                                        │ Untitled │
│ 41 │ video │ https://www.youtube.com/watch?v=TGQcDaZ56ao&list=PLS1QulWo1RIa7D1O6skqDQ-JZ1GGHKK-K&in… │ Untitled │
│ 42 │  pdf  │                                        no_url_43                                        │ Untitled │
╰────┴───────┴─────────────────────────────────────────────────────────────────────────────────────────┴──────────╯

...

Content title updated successfully for content ID 46.


Enter new title for content ID 47 (current title: Untitled): Lesson 6: Color Spaces and Conversions


Content title updated successfully for content ID 47.


Enter new title for content ID 48 (current title: Untitled): Lesson 6: Color Spaces and Conversions pdf


Content title updated successfully for content ID 48.

### **4.2.5 Update Content of course 5**

In [ ]:
display_courses_and_update_titles(db_conn, cursor)

                                            📚 Available Courses                                             
╭──────────────────────────────────────┬────────────────────────────────────────────────────────────────────╮
│                  ID                  │ Title                                                              │
├──────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │ Introduction to Computer Vision and Image Processing               │
│ 4c53688f-430a-4d8c-b21c-e796256b46da │ Mastering Natural Language Processing (NLP) with Python            │
│ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │ Machine Learning Specialization: Mastering the Art of Data Science │
│ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │ OpenCV Python Tutorial For Beginners                               │
│ fdd32b6c-cad7-4546-b958-41bf41429b3f │ Introduction to python                                             │
╰──────────────────────────────────────┴────────────────────────────────────────────────────────────────────╯


Enter the Course ID you want to update: 4c53688f-430a-4d8c-b21c-e796256b46da


         📚 Chapters for Selected Course         
╭────┬──────────────────────────────────────────╮
│ ID │ Title                                    │
├────┼──────────────────────────────────────────┤
│ 10 │ Chapter 1: Foundations of NLP            │
│ 11 │ Chapter 2: Core NLP Tasks and Models     │
│ 12 │ Chapter 3: Advanced NLP and Applications │
╰────┴──────────────────────────────────────────╯

🔖 Chapter: Chapter 1: Foundations of NLP

                               📦 Content for Chapter: Chapter 1: Foundations of NLP                               
╭────┬───────┬─────────────────────────────────────────────────────────────────────────────────────────┬──────────╮
│ ID │ Type  │                                           URL                                           │ Title    │
├────┼───────┼─────────────────────────────────────────────────────────────────────────────────────────┼──────────┤
│ 49 │ video │   https://www.youtube.com/watch?v=R-AG4-qZs1A&list=PLeo1K3hjS3uuvuAXhYjV2lMEShq2UYSwX   │ Untitled │
│ 50 │  pdf  │                                        no_url_51                                        │ Untitled │
│ 51 │ video │ https://www.youtube.com/watch?v=3y2-IaBeIs0&list=PLeo1K3hjS3uuvuAXhYjV2lMEShq2UYSwX&in… │ Untitled │
│ 52 │  pdf  │                                        no_url_52                                        │ Untitled │
│ 53 │ video │ https://www.youtube.com/watch?v=lK9gx4q_vfI&list=PLeo1K3hjS3uuvuAXhYjV2lMEShq2UYSwX&in… │ Untitled │
│ 54 │  pdf  │                                        no_url_53                                        │ Untitled │
╰────┴───────┴─────────────────────────────────────────────────────────────────────────────────────────┴──────────╯

...

Content title updated successfully for content ID 64.


Enter new title for content ID 65 (current title: Untitled): Lesson 9: Building and Deploying NLP Applications


Content title updated successfully for content ID 65.


Enter new title for content ID 66 (current title: Untitled): Lesson 9: Building and Deploying NLP Applications pdf


Content title updated successfully for content ID 66.

## **4.2.2 Update Total Hours of all the courses**

In [ ]:
def update_total_hours_for_courses(cursor, conn):
    # Update total_hours for all courses based on the sum of content durations
    update_query = '''
    UPDATE courses c
    SET c.total_hours = (
        SELECT SUM(content.duration) / 60  -- Sum of durations in hours
        FROM chapter_content cc
        JOIN content ON cc.content_ID = content.content_ID
        WHERE cc.chapter_ID IN (
            SELECT chapter_ID FROM course_chapters WHERE course_ID = c.course_ID
        )
    )
    WHERE EXISTS (
        SELECT 1
        FROM course_chapters
        WHERE course_ID = c.course_ID
    );
    '''

    try:
        cursor.execute(update_query)
        conn.commit()
        print("✅ Total hours updated successfully for all courses.")
    except Exception as e:
        print(f"❌ Error updating total hours: {e}")
        conn.rollback()

In [ ]:
update_total_hours_for_courses(cursor, db_conn)

✅ Total hours updated successfully for all courses.


## **4.3 Display Content Details**

In [ ]:
def display_course_details(cursor, course_id):
    console = Console()

    # Get course details including instructor and other fields
    cursor.execute("""
        SELECT course_title, description, category, level, price, thumbnail_url,
               total_hours, what_you_will_learn, who_this_course_is_for, instructor_id
        FROM courses WHERE course_ID = %s
    """, (course_id,))

    course_details = cursor.fetchone()

    if not course_details:
        console.print("⚠️ [bold red]Course not found.[/bold red]")
        return

    title, description, category, level, price, thumbnail_url, total_hours, what_you_will_learn, who_this_course_is_for, instructor_id = course_details

    # Display course details in a rich table
    course_table = Table(title=f"📚 [bold green]{title}[/bold green] Course Details", box=box.ROUNDED)
    course_table.add_column("Property", justify="right", style="cyan", no_wrap=True)
    course_table.add_column("Value", justify="left", style="magenta")
    course_table.add_row("Description", description)
    course_table.add_row("Category", category)
    course_table.add_row("Level", level)
    course_table.add_row("Price", f"${price}")
    course_table.add_row("Total Hours", f"{total_hours} hours")
    course_table.add_row("What You'll Learn", what_you_will_learn)
    course_table.add_row("Who This Course Is For", who_this_course_is_for)
    course_table.add_row("Thumbnail URL", thumbnail_url if thumbnail_url else "Not Available")

    # Get instructor details
    cursor.execute("SELECT full_name FROM users WHERE user_ID = %s", (instructor_id,))
    instructor_name = cursor.fetchone()[0] if instructor_id else "Unknown"
    course_table.add_row("Instructor", instructor_name)

    console.print(course_table)

    # Get all chapters for the course and their position
    cursor.execute("""
        SELECT chapters.chapter_ID, chapters.title, chapters.description, course_chapters.position
        FROM chapters
        JOIN course_chapters ON chapters.chapter_ID = course_chapters.chapter_ID
        WHERE course_chapters.course_ID = %s
        ORDER BY course_chapters.position
    """, (course_id,))

    chapters = cursor.fetchall()

    if not chapters:
        console.print("⚠️ [bold red]No chapters found for this course.[/bold red]")
        return

    # Show the connection between course and chapters in a table
    chapter_table = Table(title="📚 [bold yellow]Chapters for this Course[/bold yellow]", box=box.ROUNDED)
    chapter_table.add_column("Position", style="cyan", justify="right")
    chapter_table.add_column("Chapter ID", style="magenta", justify="center")
    chapter_table.add_column("Title", justify="left")
    chapter_table.add_column("Description", justify="left")
    chapter_table.add_column("Content Count", justify="center")

    for idx, (chapter_id, chapter_title, chapter_desc, position) in enumerate(chapters, 1):
        # Get the number of contents for the chapter
        cursor.execute("""
            SELECT COUNT(*) FROM chapter_content WHERE chapter_ID = %s
        """, (chapter_id,))
        content_count = cursor.fetchone()[0]

        chapter_table.add_row(str(position), str(chapter_id), chapter_title, chapter_desc, str(content_count))

    console.print(chapter_table)

    # For each chapter, list its content with their durations
    for idx, (chapter_id, chapter_title, chapter_desc, position) in enumerate(chapters, 1):
        console.print(f"\n🔖 [bold blue]Chapter {idx}: {chapter_title} (Position: {position})[/bold blue]")
        console.print(f"   📜 Description: {chapter_desc}")

        # Get content for the chapter (including content_title)
        cursor.execute("""
            SELECT content.content_ID, content.content_type, content.content_url, content.content_title, content.description, content.duration
            FROM content
            JOIN chapter_content ON content.content_ID = chapter_content.content_ID
            WHERE chapter_content.chapter_ID = %s
            ORDER BY chapter_content.position
        """, (chapter_id,))

        content = cursor.fetchall()

        if not content:
            console.print(f"   ⚠️ [bold red]No content found for this chapter.[/bold red]")
        else:
            content_table = Table(title=f"📦 Content for [bold cyan]{chapter_title}[/bold cyan]", box=box.ROUNDED)
            content_table.add_column("Content ID", style="yellow", justify="center")
            content_table.add_column("Type", style="yellow", justify="center")
            content_table.add_column("URL", style="magenta", justify="center")
            content_table.add_column("Title", style="green", justify="left")
            content_table.add_column("Description", style="green")
            content_table.add_column("Duration", style="red", justify="center")

            for content_idx, (content_id, content_type, content_url, content_title, content_desc, duration) in enumerate(content, 1):
                content_table.add_row(str(content_id), content_type.upper(), content_url, content_title, content_desc, f"{duration} minutes")

            console.print(content_table)


In [ ]:
# 1️⃣ Show available courses and allow user to choose one
def choose_and_display_course(cursor):
    cursor.execute("SELECT course_ID, course_title FROM courses")
    courses = cursor.fetchall()

    if not courses:
        print("⚠️ No courses found.")
        return None

    print("\n📚 Available Courses:")
    for idx, (cid, title) in enumerate(courses, 1):
        print(f"{idx}. {title} (ID: {cid})")

    while True:
        try:
            choice = int(input("➡️ Choose a course by number: "))
            if 1 <= choice <= len(courses):
                course_id = courses[choice - 1][0]
                print(f"\n📖 Displaying details for course: {courses[choice - 1][1]} (ID: {course_id})")
                display_course_details(cursor, course_id)  # Call to display full course info
                return
            else:
                print("❌ Invalid number.")
        except ValueError:
            print("❌ Please enter a valid number.")

### **4.3.1 Display Content Details Course 1**

In [ ]:
choose_and_display_course(cursor)


📚 Available Courses:
1. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
2. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)
➡️ Choose a course by number: 1

📖 Displaying details for course: Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)


               📚 Machine Learning Specialization: Mastering the Art of Data Science Course Details                
╭────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮
│               Property │ Value                                                                                  │
├────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│            Description │ Welcome to the Machine Learning Specialization! This comprehensive program is designed │
│                        │ to take you from a beginner to an expert in the field of machine learning. Whether     │
│                        │ you're a data enthusiast, aspiring data scientist, or someone looking to expand your   │
│                        │ knowledge in AI and machine learning, this course will provide you with the necessary  │
│                        │ skills and hands-on experience to build impactful machine learning models.  Through a  │
│                        │ series of lessons and projects, you'll explore a wide range of machine learning        │
│                        │ techniques and algorithms, from supervised learning and regression models to           │
│                        │ unsupervised learning and deep learning. You will gain a deep understanding of how to  │
│                        │ apply these methods in real-world applications, analyze data, and evaluate model       │
│                        │ performance.                                                                           │
│               Category │ Machine Learning                                                                       │
│                  Level │ Beginner                                                                               │
│                  Price │ $0.0                                                                                   │
│            Total Hours │ 1.2833333333333334 hours                                                               │
│      What You'll Learn │ Aspiring data scientists or machine learning engineers.  Professionals in AI, business │
│                        │ intelligence, or analytics.  Anyone interested in learning how machine learning        │
│                        │ algorithms work and how to apply them.                                                 │
│ Who This Course Is For │ Hands-on Projects: Practical applications and case studies to help you build a strong  │
│                        │ portfolio.  Expert Instructors: Learn from leading experts in the field who provide    │
│                        │ clear, step-by-step guidance.  In-Demand Skills: Equip yourself with the skills that   │
│                        │ are highly sought after in the growing field of data science and AI.                   │
│          Thumbnail URL │ https://www.google.com/search?sca_esv=0eb164f915fa3037&sxsrf=AHTn8zoT_2kwsa4t1EnXzdfR… │
│             Instructor │ Mohamed Hatem                                                                          │
╰────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯

                                            📚 Chapters for this Course                                            
╭──────────┬────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────╮
│ Position │ Chapter ID │ Title                              │ Description                        │ Content Count │
├──────────┼────────────┼────────────────────────────────────┼────────────────────────────────────┼───────────────┤
│        1 │     1      │ Chapter 1: Introduction to Machine │  This chapter introduces the       │       6       │
│          │            │ Learning                           │ fundamental concepts of machine    │               │
│          │            │                                    │ learning, its importance, and its  │               │
│          │            │                                    │ widespread applications across     │               │
│          │            │                                    │ various industries. Learn why      │               │
│          │            │                                    │ machine learning is vital for      │               │
│          │            │                                    │ handling massive data, automating  │               │
│          │            │                                    │ tasks, and enabling better         │               │
│          │            │                                    │ decision-making. It also           │               │
│          │            │                                    │ introduces Artificial General      │               │
│          │            │                                    │ Intelligence (AGI) and its         │               │
│          │            │                                    │ potential future impact.           │               │
│        2 │     2      │ Chapter 2: What is Machine         │ In this chapter, we define machine │       6       │
│          │            │ Learning?                          │ learning in simple terms and break │               │
│          │            │                                    │ down its core concepts. Explore    │               │
│          │            │                                    │ the three main types of machine    │               │
│          │            │                                    │ learning algorithms: supervised,   │               │
│          │            │                                    │ unsupervised, and reinforcement    │               │
│          │            │                                    │ learning. Discover real-world      │               │
│          │            │                                    │ examples of how these algorithms   │               │
│          │            │                                    │ are applied in various fields.     │               │
│        3 │     3      │ Chapter 3: Supervised Learning –   │ This chapter provides a detailed   │       6       │
│          │            │ Part 1                             │ exploration of supervised          │               │
│          │            │                                    │ learning, a widely used machine    │               │
│          │            │                                    │ learning technique. Understand how │               │
│          │            │                                    │ supervised learning models are     │               │
│          │            │                                    │ trained on labeled data, and learn │               │
│          │            │                                    │ about the two primary types of     │               │
│          │            │                                    │ supervised learning tasks:         │               │
│          │            │                                    │ regression and classification. Key │               │
│          │            │                                

🔖 Chapter 1: Chapter 1: Introduction to Machine Learning (Position: 1)

📜 Description:  This chapter introduces the fundamental concepts of machine learning, its importance, and its 
widespread applications across various industries. Learn why machine learning is vital for handling massive data, 
automating tasks, and enabling better decision-making. It also introduces Artificial General Intelligence (AGI) and
its potential future impact.

...

📜 Description: In this chapter, we define machine learning in simple terms and break down its core concepts. 
Explore the three main types of machine learning algorithms: supervised, unsupervised, and reinforcement learning. 
Discover real-world examples of how these algorithms are applied in various fields.

                                📦 Content for Chapter 2: What is Machine Learning?                                
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     7      │ VIDEO │ https://www.youtube.co… │ Lesson 4: Supervised    │ In this lesson, we      │ 15.0 minutes │
│            │       │                         │ Learning Part 2 video   │ explore the most        │              │
│            │       │                         │                         │ popular supervised      │              │
│            │       │                         │                         │ learning algorithms —   │              │
│            │       │                         │                         │ like Linear Regression, │              │
│            │       │                         │                         │ Decision Trees, and SVM │              │
│            │       │                         │                         │ — and guide you on how  │              │
│            │       │                         │                         │ to choose the right one │              │
│            │       │                         │                         │ for your data.          │              │
│     8      │  PDF  │       temp_url_4        │ Lesson 4: Supervised    │ This lesson dives into  │ 1.0 minutes  │
│            │       │                         │ Learning Part 2 pdf     │ key supervised learning │              │
│            │       │                         │                         │ algorithms, comparing   │              │
│            │       │                         │                         │ their use cases,        │              │
│            │       │                         │                         │ strengths, and          │              │
│            │       │                         │                         │ limitations, and        │              │
│            │       │                         │                         │ offering tips on        │              │
│            │       │                         │                         │ selecting the best fit  │              │
│            │       │                         │                         │ for your machine        │              │
│            │       │                         │                         │ learning problem.       │              │
│     9      │ VIDEO │ https://www.youtube.co… │ Lesson 5: Unsupervised  │ Discover the world of   │ 5.0 minutes  │
│            │       │                         │ Learning – Part 1 video │ unsupervised learning!  │              │
│            │       │                         │                         │ This lesson introduces  │              │
│            │       │                         │                         │ clustering and explores │              │
│            │       │                         │                         │ how it powers           │              │
│            │       │                         │                         │ real-world applications │              │
│            │       │                         │                         │ like Google News        │              │
│            │       │                         │                         │ grouping and DNA        │              │
│            │       │                         │                         │ microarray analysis.    │              │
│     10     │  PDF  │       temp_url_5        │ Lesson 5: Unsupervised  │ This lesson explains    │ 1.0 minutes  │
│            │       │                         │ Learning – Part 1 pdf   │ unsupervised learning   │              │
│            │       │                         │         

🔖 Chapter 3: Chapter 3: Supervised Learning – Part 1 (Position: 3)

📜 Description: This chapter provides a detailed exploration of supervised learning, a widely used machine 
learning technique. Understand how supervised learning models are trained on labeled data, and learn about the two 
primary types of supervised learning tasks: regression and classification. Key concepts such as input features, 
output labels, training data, and testing data are also covered.

                              📦 Content for Chapter 3: Supervised Learning – Part 1                               
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     13     │ VIDEO │ https://www.youtube.co… │ Lesson 7: Linear        │ Get introduced to       │ 5.0 minutes  │
│            │       │                         │ Regression Model – Part │ Linear Regression — one │              │
│            │       │                         │ 1 video                 │ of the most essential   │              │
│            │       │                         │                         │ models in machine       │              │
│            │       │                         │                         │ learning. Learn how it  │              │
│            │       │                         │                         │ works, where it’s used, │              │
│            │       │                         │                         │ and the key concepts    │              │
│            │       │                         │                         │ like weights, bias, and │              │
│            │       │                         │                         │ loss functions.         │              │
│     14     │  PDF  │       temp_url_7        │ Lesson 7: Linear        │ This lesson covers the  │ 1.0 minutes  │
│            │       │                         │ Regression Model – Part │ fundamentals of Linear  │              │
│            │       │                         │ 1 pdf                   │ Regression. Understand  │              │
│            │       │                         │                         │ its applications, the   │              │
│            │       │                         │                         │ underlying equation,    │              │
│            │       │                         │                         │ and key terminology     │              │
│            │       │                         │                         │ including features,     │              │
│            │       │                         │                         │ targets, weights, and   │              │
│            │       │                         │                         │ gradient descent — all  │              │
│            │       │                         │                         │ with real-world         │              │
│            │       │                         │                         │ examples.               │              │
│     15     │ VIDEO │ https://www.youtube.co… │ Lesson 8: Linear        │ Take your linear        │ 7.0 minutes  │
│            │       │                         │ Regression Model – Part │ regression skills to    │              │
│            │       │                         │ 2 video                 │ the next level! Learn   │              │
│            │       │                         │                         │ how to train, test, and │              │
│            │       │                         │                         │ evaluate a linear model │              │
│            │       │                         │                         │ using Python and        │              │
│            │       │                         │                         │ scikit-learn — complete │              │
│            │       │                         │                         │ with hands-on code and  │              │
│            │       │                         │                         │ clear explanations of   │              │
│            │       │                         │                         │ MSE and R².             │              │
│     16     │  PDF  │       temp_url_8        │ Lesson 8

### **4.3.2 Display Content Details Course 2**

In [ ]:
choose_and_display_course(cursor)


📚 Available Courses:
1. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
2. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)
➡️ Choose a course by number: 2

📖 Displaying details for course: Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)


                                     📚 Introduction to python Course Details                                      
╭────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮
│               Property │ Value                                                                                  │
├────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│            Description │ Welcome to the 100 Days of Code - The Complete Python Pro Bootcamp, the only course    │
│                        │ you need to learn to code with Python. With over 500,000 5 STAR reviews and a 4.8      │
│                        │ average, my courses are some of the HIGHEST RATED courses in the history of Udemy!     │
│               Category │ Programming Language                                                                   │
│                  Level │ Intermediate                                                                           │
│                  Price │ $0.0                                                                                   │
│            Total Hours │ 0.7666666666666667 hours                                                               │
│      What You'll Learn │ You will master the Python programming language by building 100 unique projects over   │
│                        │ 100 days. You will learn automation, game, app and web development, data science and   │
│                        │ machine learning all using Python.                                                     │
│ Who This Course Is For │ No programming experience needed - I'll teach you everything you need to know A Mac or │
│                        │ PC computer with access to the internet No paid software required - I'll teach you how │
│                        │ to use PyCharm, Jupyter Notebooks and Google Colab I'll walk you through, step-by-step │
│                        │ how to get all the software installed and set up                                       │
│          Thumbnail URL │ https://www.google.com/search?sca_esv=0c50508b885de737&sxsrf=AHTn8zqvp2gvnvkOUWaeQv84… │
│             Instructor │ Ahmed Mamdouh                                                                          │
╰────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯

                                            📚 Chapters for this Course                                            
╭──────────┬────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────╮
│ Position │ Chapter ID │ Title                              │ Description                        │ Content Count │
├──────────┼────────────┼────────────────────────────────────┼────────────────────────────────────┼───────────────┤
│        1 │     4      │ Chapter 1  : Working with          │ In this chapter, you will learn    │       6       │
│          │            │ variables in python                │ the foundational concept of        │               │
│          │            │                                    │ variables in Python programming.   │               │
│          │            │                                    │ We’ll explore how to create,       │               │
│          │            │                                    │ assign, and update variables, as   │               │
│          │            │                                    │ well as understand the different   │               │
│          │            │                                    │ data types Python supports.        │               │
│          │            │                                    │ Through practical examples, you'll │               │
│          │            │                                    │ grasp the importance of naming     │               │
│          │            │                                    │ conventions, dynamic typing, and   │               │
│          │            │                                    │ type conversion. This chapter sets │               │
│          │            │                                    │ the stage for building more        │               │
│          │            │                                    │ complex programs by establishing a │               │
│          │            │                                    │ solid understanding of how Python  │               │
│          │            │                                    │ stores and manages data.           │               │
│        2 │     5      │ Chapter 2 : Control flow and       │  This chapter introduces the       │       4       │
│          │            │ Operators                          │ essential building blocks for      │               │
│          │            │                                    │ decision-making and logic in       │               │
│          │            │                                    │ Python: control flow and           │               │
│          │            │                                    │ operators. You’ll learn how to use │               │
│          │            │                                    │ conditional statements (if, elif,  │               │
│          │            │                                    │ else) and loops (for, while) to    │               │
│          │            │                                    │ control the execution of your code │               │
│          │            │                                    │ based on specific conditions.      │               │
│          │            │                                    │ Additionally, we’ll cover Python’s │               │
│          │            │                                    │ various operators—arithmetic,      │               │
│          │            │                                    │ comparison, logical, and           │               │
│          │            │                                    │ assignment—to perform operations   │               │
│          │            │                                    │ on data and build powerful         │               │
│          │            │                                    │ expressions. By the end of this    │               │
│          │            │                                

🔖 Chapter 1: Chapter 1  : Working with variables in python (Position: 1)

📜 Description: In this chapter, you will learn the foundational concept of variables in Python programming. 
We’ll explore how to create, assign, and update variables, as well as understand the different data types Python 
supports. Through practical examples, you'll grasp the importance of naming conventions, dynamic typing, and type 
conversion. This chapter sets the stage for building more complex programs by establishing a solid understanding of
how Python stores and manages data.

                           📦 Content for Chapter 1  : Working with variables in python                            
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     19     │ VIDEO │ https://www.youtube.co… │ Lesson 1: Introduction  │ Get started with        │ 12.0 minutes │
│            │       │                         │ and What's Python video │ Python, one of the most │              │
│            │       │                         │                         │ versatile and           │              │
│            │       │                         │                         │ beginner-friendly       │              │
│            │       │                         │                         │ programming languages.  │              │
│            │       │                         │                         │ Learn about its         │              │
│            │       │                         │                         │ features, uses, and the │              │
│            │       │                         │                         │ applications built with │              │
│            │       │                         │                         │ Python that shape the   │              │
│            │       │                         │                         │ world today.            │              │
│     20     │  PDF  │       temp_url_21       │ Lesson 1: Introduction  │ In this lesson, you'll  │ 1.0 minutes  │
│            │       │                         │ and What's Pythonpdf    │ explore what Python is, │              │
│            │       │                         │                         │ why it's an excellent   │              │
│            │       │                         │                         │ choice for developers,  │              │
│            │       │                         │                         │ and the vast array of   │              │
│            │       │                         │                         │ applications it powers, │              │
│            │       │                         │                         │ from web development to │              │
│            │       │                         │                         │ machine learning.       │              │
│     21     │ VIDEO │ https://www.youtube.co… │ Lesson 2: Syntax and    │ Learn the basic syntax  │ 10.0 minutes │
│            │       │                         │ Your First App video    │ of Python and build     │              │
│            │       │                         │                         │ your first Python       │              │
│            │       │                         │                         │ application — a simple  │              │
│            │       │                         │                         │ calculator that         │              │
│            │       │                         │                         │ performs basic          │              │
│            │       │                         │                         │ arithmetic operations   │              │
│            │       │                         │                         │ based on user input.    │              │
│     22     │  PDF  │       temp_url_22       │ Lesson 2: Syntax and    │ In this lesson, explore │ 1.0 minutes  │
│            │       │                         │ Your First App pdf      │ the fundamental rules   │              │
│            │       │                         │                         │ of Python syntax, from  │              │
│            │       │                         │                         │ case sensitivity to     │              │
│            │       │                         │         

🔖 Chapter 2: Chapter 2 : Control flow and Operators  (Position: 2)

📜 Description:  This chapter introduces the essential building blocks for decision-making and logic in Python: 
control flow and operators. You’ll learn how to use conditional statements (if, elif, else) and loops (for, while) 
to control the execution of your code based on specific conditions. Additionally, we’ll cover Python’s various 
operators—arithmetic, comparison, logical, and assignment—to perform operations on data and build powerful 
expressions. By the end of this chapter, you'll be able to write programs that make decisions, repeat actions, and 
evaluate conditions with confidence.

                               📦 Content for Chapter 2 : Control flow and Operators                               
╭────────────┬───────┬─────────────────────────┬──────────────────────────┬─────────────────────────┬─────────────╮
│ Content ID │ Type  │           URL           │ Title                    │ Description             │  Duration   │
├────────────┼───────┼─────────────────────────┼──────────────────────────┼─────────────────────────┼─────────────┤
│     25     │ VIDEO │ https://www.youtube.co… │ Lesson 4: Control Flow   │ Learn how to control    │ 5.0 minutes │
│            │       │                         │ video                    │ the direction of your   │             │
│            │       │                         │                          │ Python programs using   │             │
│            │       │                         │                          │ conditional statements  │             │
│            │       │                         │                          │ (if, elif, else), loops │             │
│            │       │                         │                          │ (for, while), and       │             │
│            │       │                         │                          │ control statements      │             │
│            │       │                         │                          │ (break, continue,       │             │
│            │       │                         │                          │ pass).                  │             │
│     26     │  PDF  │       temp_url_24       │ Lesson 4: Control Flow   │ Master the basics of    │ 1.0 minutes │
│            │       │                         │ pdf                      │ program control in      │             │
│            │       │                         │                          │ Python with conditional │             │
│            │       │                         │                          │ statements, loops, and  │             │
│            │       │                         │                          │ control statements.     │             │
│            │       │                         │                          │ These tools will help   │             │
│            │       │                         │                          │ you create programs     │             │
│            │       │                         │                          │ that respond to         │             │
│            │       │                         │                          │ different conditions    │             │
│            │       │                         │                          │ and repeat tasks        │             │
│            │       │                         │                          │ effectively.            │             │
│     27     │ VIDEO │ https://www.youtube.co… │ Lesson 5: Functions      │ Master the power of     │ 7.0 minutes │
│            │       │                         │ video                    │ functions in Python to  │             │
│            │       │                         │                          │ make your code reusable │             │
│            │       │                         │                          │ and organized. Learn    │             │
│            │       │                         │                          │ how to define           │             │
│            │       │                         │                          │ functions, pass         │             │
│            │       │                         │                          │ parameters, return      │             │
│            │       │                         │                          │ values, and use both    │             │
│            │       │                         │                          │ built-in and            │             │
│            │       │                         │                          │ user-defined functions. │             │
│     28     │  PDF  │       temp_url_25       │ Lesson 5

### **4.3.3 Display Content Details Course 3**

In [ ]:
choose_and_display_course(cursor)


📚 Available Courses:
1. Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)
2. Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)
3. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
4. OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)
5. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)
➡️ Choose a course by number: 1

📖 Displaying details for course: Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)


                      📚 Introduction to Computer Vision and Image Processing Course Details                       
╭────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮
│               Property │ Value                                                                                  │
├────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│            Description │ This course is designed to introduce learners to the fascinating world of computer     │
│                        │ vision and digital image processing. You'll start with the foundational concepts of    │
│                        │ how machines understand visual information, then dive deeper into the techniques used  │
│                        │ to process, filter, and extract features from images. With a mix of theory and         │
│                        │ practical knowledge, this course prepares you for advanced topics in AI, robotics, and │
│                        │ visual recognition systems.  Throughout the course, you'll gain hands-on understanding │
│                        │ of how images are represented digitally, how to enhance them through filtering, and    │
│                        │ how to extract meaningful information through edge detection. By the end, you’ll be    │
│                        │ equipped with essential skills for any computer vision project.                        │
│               Category │ Computer Vision                                                                        │
│                  Level │ Intermediate                                                                           │
│                  Price │ $0.0                                                                                   │
│            Total Hours │ 3.216666666666667 hours                                                                │
│      What You'll Learn │ The fundamentals of computer vision and its real-world applications  The basic steps   │
│                        │ involved in image analysis  How digital images are represented and processed  Common   │
│                        │ color models and image types  Image filtering techniques for noise reduction and       │
│                        │ enhancement  How to perform edge detection and extract key visual features  The        │
│                        │ practical importance of these techniques in AI and machine learning                    │
│ Who This Course Is For │ Beginners and students interested in computer vision and image processing  Aspiring AI │
│                        │ and machine learning engineers  Robotics enthusiasts working with vision-based systems │
│                        │ Developers seeking to understand visual data analysis  Anyone curious about how        │
│                        │ machines “see” and interpret the world                                                 │
│          Thumbnail URL │ https://www.google.com/search?sca_esv=b4544fb49b72c327&sxsrf=AHTn8zpGKf-emyhW4j3F28al… │
│             Instructor │ Omar Mohamed Nagib                                                                     │
╰────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯

                                            📚 Chapters for this Course                                            
╭──────────┬────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────╮
│ Position │ Chapter ID │ Title                              │ Description                        │ Content Count │
├──────────┼────────────┼────────────────────────────────────┼────────────────────────────────────┼───────────────┤
│        1 │     6      │ Chapter 1: Foundations of Computer │ This chapter builds the groundwork │       4       │
│          │            │ Vision and Image Processing        │ for working with visual data by    │               │
│          │            │                                    │ introducing the fundamental        │               │
│          │            │                                    │ concepts of computer vision and    │               │
│          │            │                                    │ digital image processing. You’ll   │               │
│          │            │                                    │ explore how machines see images,   │               │
│          │            │                                    │ how images are structured, and how │               │
│          │            │                                    │ to prepare them for analysis using │               │
│          │            │                                    │ basic techniques.                  │               │
│        2 │     7      │ Chapter 2: Image Filtering and     │ In this chapter, you'll dive into  │       4       │
│          │            │ Edge Detection                     │ key techniques used to enhance     │               │
│          │            │                                    │ images and extract important       │               │
│          │            │                                    │ features. You’ll explore spatial   │               │
│          │            │                                    │ filtering methods to sharpen or    │               │
│          │            │                                    │ smooth images and understand how   │               │
│          │            │                                    │ edge detection reveals object      │               │
│          │            │                                    │ boundaries. These are critical     │               │
│          │            │                                    │ steps in preparing images for      │               │
│          │            │                                    │ recognition and analysis.          │               │
╰──────────┴────────────┴────────────────────────────────────┴────────────────────────────────────┴───────────────╯

🔖 Chapter 1: Chapter 1: Foundations of Computer Vision and Image Processing (Position: 1)

📜 Description: This chapter builds the groundwork for working with visual data by introducing the fundamental 
concepts of computer vision and digital image processing. You’ll explore how machines see images, how images are 
structured, and how to prepare them for analysis using basic techniques.

                   📦 Content for Chapter 1: Foundations of Computer Vision and Image Processing                   
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     29     │ VIDEO │ https://www.youtube.co… │ Lesson 1: Introduction  │ This lesson introduces  │ 45.0 minutes │
│            │       │                         │ to Computer Vision      │ the basics of computer  │              │
│            │       │                         │                         │ vision, guiding you     │              │
│            │       │                         │                         │ through the core steps  │              │
│            │       │                         │                         │ of image analysis and   │              │
│            │       │                         │                         │ preprocessing. It sets  │              │
│            │       │                         │                         │ the foundation for      │              │
│            │       │                         │                         │ working with visual     │              │
│            │       │                         │                         │ data in AI              │              │
│            │       │                         │                         │ applications.           │              │
│     30     │  PDF  │        no_url_1         │ Lesson 1: Introduction  │ This lesson introduces  │ 1.0 minutes  │
│            │       │                         │ to Computer Vision pdf  │ the basics of computer  │              │
│            │       │                         │                         │ vision, guiding you     │              │
│            │       │                         │                         │ through the core steps  │              │
│            │       │                         │                         │ of image analysis and   │              │
│            │       │                         │                         │ preprocessing. It sets  │              │
│            │       │                         │                         │ the foundation for      │              │
│            │       │                         │                         │ working with visual     │              │
│            │       │                         │                         │ data in AI              │              │
│            │       │                         │                         │ applications.           │              │
│     31     │ VIDEO │ https://www.youtube.co… │ Lesson 2:  Introduction │ This lesson introduces  │ 55.0 minutes │
│            │       │                         │ to Digital Image        │ the basics of computer  │              │
│            │       │                         │ Processing              │ vision, guiding you     │              │
│            │       │                         │                         │ through the core steps  │              │
│            │       │                         │                         │ of image analysis and   │              │
│            │       │                         │                         │ preprocessing. It sets  │              │
│            │       │                         │                         │ the foundation for      │              │
│            │       │                         │                         │ working with visual     │              │
│            │       │                         │                         │ data in AI              │              │
│            │       │                         │                         │ applications.           │              │
│     32     │  PDF  │        no_url_2         │ Lesson 2

🔖 Chapter 2: Chapter 2: Image Filtering and Edge Detection (Position: 2)

📜 Description: In this chapter, you'll dive into key techniques used to enhance images and extract important 
features. You’ll explore spatial filtering methods to sharpen or smooth images and understand how edge detection 
reveals object boundaries. These are critical steps in preparing images for recognition and analysis.

                           📦 Content for Chapter 2: Image Filtering and Edge Detection                            
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     33     │ VIDEO │ https://www.youtube.co… │ Lesson 3: Image         │ This lesson covers the  │ 41.0 minutes │
│            │       │                         │ Filtering Techniques    │ most common image       │              │
│            │       │                         │                         │ filtering methods used  │              │
│            │       │                         │                         │ to improve image        │              │
│            │       │                         │                         │ quality and highlight   │              │
│            │       │                         │                         │ specific features.      │              │
│     34     │  PDF  │        no_url_3         │ Lesson 3: Image         │ This lesson focuses on  │ 1.0 minutes  │
│            │       │                         │ Filtering Techniques    │ detecting object        │              │
│            │       │                         │ pdf                     │ boundaries and          │              │
│            │       │                         │                         │ extracting essential    │              │
│            │       │                         │                         │ features from images    │              │
│            │       │                         │                         │ using various edge      │              │
│            │       │                         │                         │ detection algorithms.   │              │
│     35     │ VIDEO │ https://www.youtube.co… │ Lesson 4: Edge          │ This lesson focuses on  │ 48.0 minutes │
│            │       │                         │ Detection and Feature   │ detecting object        │              │
│            │       │                         │ Extraction              │ boundaries and          │              │
│            │       │                         │                         │ extracting essential    │              │
│            │       │                         │                         │ features from images    │              │
│            │       │                         │                         │ using various edge      │              │
│            │       │                         │                         │ detection algorithms.   │              │
│     36     │  PDF  │        no_url_4         │ Lesson 4: Edge          │ This lesson focuses on  │ 1.0 minutes  │
│            │       │                         │ Detection and Feature   │ detecting object        │              │
│            │       │                         │ Extraction pdf          │ boundaries and          │              │
│            │       │                         │                         │ extracting essential    │              │
│            │       │                         │                         │ features from images    │              │
│            │       │                         │                         │ using various edge      │              │
│            │       │                         │                         │ detection algorithms.   │              │
╰────────────┴───────┴─────────────────────────┴─────────────────────────┴─────────────────────────┴──────────────╯

### **4.3.4 Display Content Details Course 4**

In [ ]:
choose_and_display_course(cursor)


📚 Available Courses:
1. Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)
2. Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)
3. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
4. OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)
5. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)
➡️ Choose a course by number: 4

📖 Displaying details for course: OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)


                              📚 OpenCV Python Tutorial For Beginners Course Details                               
╭────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮
│               Property │ Value                                                                                  │
├────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│            Description │ This hands-on course introduces you to OpenCV, the most popular computer vision        │
│                        │ library, using the Python programming language. You'll start by learning how to read,  │
│                        │ display, and manipulate images, then progress to more advanced topics like drawing     │
│                        │ shapes, working with videos, and performing real-time image operations.  Through       │
│                        │ practical examples and beginner-friendly lessons, this course makes it easy to get     │
│                        │ started with OpenCV—even if you’ve never worked with images before.                    │
│               Category │ OpenCV                                                                                 │
│                  Level │ Advanced                                                                               │
│                  Price │ $0.0                                                                                   │
│            Total Hours │ 1.35 hours                                                                             │
│      What You'll Learn │ How to install and set up OpenCV in Python  How to read, display, and save images      │
│                        │ Drawing shapes, adding text, and basic manipulations  Working with videos and          │
│                        │ accessing webcams  Image transformation and filtering  Real-time processing using      │
│                        │ OpenCV                                                                                 │
│ Who This Course Is For │ Beginners with no prior experience in computer vision  Python developers looking to    │
│                        │ expand into image processing  AI and ML learners seeking practical OpenCV skills       │
│                        │ Students working on vision-related academic projects  Anyone curious about building    │
│                        │ computer vision applications                                                           │
│          Thumbnail URL │ https://www.google.com/search?sca_esv=b4544fb49b72c327&sxsrf=AHTn8zoJ6iLIaGVYVMKbTANp… │
│             Instructor │ Ahmed Mamdouh                                                                          │
╰────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯

                                            📚 Chapters for this Course                                            
╭──────────┬────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────╮
│ Position │ Chapter ID │ Title                              │ Description                        │ Content Count │
├──────────┼────────────┼────────────────────────────────────┼────────────────────────────────────┼───────────────┤
│        1 │     8      │ Chapter 1: Getting Started with    │ In this chapter, you'll learn the  │       6       │
│          │            │ OpenCV                             │ basics of using OpenCV with        │               │
│          │            │                                    │ Python, including how to install   │               │
│          │            │                                    │ the library, read and display      │               │
│          │            │                                    │ images, and perform simple drawing │               │
│          │            │                                    │ tasks. This chapter is your first  │               │
│          │            │                                    │ step into the world of computer    │               │
│          │            │                                    │ vision.                            │               │
│        2 │     9      │ Chapter 2: Working with Videos and │ This chapter dives deeper into     │       6       │
│          │            │ Image Operations                   │ OpenCV capabilities by introducing │               │
│          │            │                                    │ video processing and real-time     │               │
│          │            │                                    │ image operations. You'll learn to  │               │
│          │            │                                    │ read video files, capture webcam   │               │
│          │            │                                    │ input, and apply simple            │               │
│          │            │                                    │ transformations to images.         │               │
╰──────────┴────────────┴────────────────────────────────────┴────────────────────────────────────┴───────────────╯

🔖 Chapter 1: Chapter 1: Getting Started with OpenCV (Position: 1)

📜 Description: In this chapter, you'll learn the basics of using OpenCV with Python, including how to install 
the library, read and display images, and perform simple drawing tasks. This chapter is your first step into the 
world of computer vision.

                               📦 Content for Chapter 1: Getting Started with OpenCV                               
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     37     │ VIDEO │ https://www.youtube.co… │ Lesson 1: Installing    │ Learn how to install    │ 6.0 minutes  │
│            │       │                         │ OpenCV and Setting Up   │ OpenCV using pip and    │              │
│            │       │                         │ the Environment         │ set up your Python      │              │
│            │       │                         │                         │ environment (IDE or     │              │
│            │       │                         │                         │ Jupyter Notebook) for   │              │
│            │       │                         │                         │ computer vision         │              │
│            │       │                         │                         │ development.            │              │
│     38     │  PDF  │        no_url_41        │ Lesson 1: Installing    │ Learn how to install    │ 1.0 minutes  │
│            │       │                         │ OpenCV and Setting Up   │ OpenCV using pip and    │              │
│            │       │                         │ the Environment pdf     │ set up your Python      │              │
│            │       │                         │                         │ environment (IDE or     │              │
│            │       │                         │                         │ Jupyter Notebook) for   │              │
│            │       │                         │                         │ computer vision         │              │
│            │       │                         │                         │ development.            │              │
│     39     │ VIDEO │ https://www.youtube.co… │ Lesson 2: Reading,      │ Explore how to load     │ 10.0 minutes │
│            │       │                         │ Displaying, and Saving  │ images using            │              │
│            │       │                         │ Images                  │ cv2.imread(), display   │              │
│            │       │                         │                         │ them using              │              │
│            │       │                         │                         │ cv2.imshow(), and save  │              │
│            │       │                         │                         │ processed images to     │              │
│            │       │                         │                         │ disk with               │              │
│            │       │                         │                         │ cv2.imwrite().          │              │
│     40     │  PDF  │        no_url_42        │ Lesson 2: Reading,      │ Explore how to load     │ 1.0 minutes  │
│            │       │                         │ Displaying, and Saving  │ images using            │              │
│            │       │                         │ Images pdf              │ cv2.imread(), display   │              │
│            │       │                         │                         │ them using              │              │
│            │       │                         │                         │ cv2.imshow(), and save  │              │
│            │       │                         │                         │ processed images to     │              │
│            │       │                         │                         │ disk with               │              │
│            │       │                         │                         │ cv2.imwrite().          │              │
│     41     │ VIDEO │ https://www.youtube.co… │ Lesson 3

🔖 Chapter 2: Chapter 2: Working with Videos and Image Operations (Position: 2)

📜 Description: This chapter dives deeper into OpenCV capabilities by introducing video processing and real-time
image operations. You'll learn to read video files, capture webcam input, and apply simple transformations to 
images.

                        📦 Content for Chapter 2: Working with Videos and Image Operations                         
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     43     │ VIDEO │ https://www.youtube.co… │ Lesson 4: Reading and   │ Learn how to capture    │ 8.0 minutes  │
│            │       │                         │ Writing Videos          │ video from files and    │              │
│            │       │                         │                         │ webcams using           │              │
│            │       │                         │                         │ cv2.VideoCapture() and  │              │
│            │       │                         │                         │ how to write video      │              │
│            │       │                         │                         │ output using            │              │
│            │       │                         │                         │ cv2.VideoWriter().      │              │
│     44     │  PDF  │        no_url_44        │ Lesson 4: Reading and   │ Learn how to capture    │ 1.0 minutes  │
│            │       │                         │ Writing Videos pdf      │ video from files and    │              │
│            │       │                         │                         │ webcams using           │              │
│            │       │                         │                         │ cv2.VideoCapture() and  │              │
│            │       │                         │                         │ how to write video      │              │
│            │       │                         │                         │ output using            │              │
│            │       │                         │                         │ cv2.VideoWriter().      │              │
│     45     │ VIDEO │ https://www.youtube.co… │ Lesson 5: Image         │ Explore practical image │ 17.0 minutes │
│            │       │                         │ Resizing, Cropping, and │ operations like         │              │
│            │       │                         │ Flipping                │ resizing                │              │
│            │       │                         │                         │ (cv2.resize()),         │              │
│            │       │                         │                         │ cropping using array    │              │
│            │       │                         │                         │ slicing, and flipping   │              │
│            │       │                         │                         │ images horizontally or  │              │
│            │       │                         │                         │ vertically with         │              │
│            │       │                         │                         │ cv2.flip().             │              │
│     46     │  PDF  │        no_url_45        │ Lesson 5: Image         │ Explore practical image │ 1.0 minutes  │
│            │       │                         │ Resizing, Cropping, and │ operations like         │              │
│            │       │                         │ Flipping pdf            │ resizing                │              │
│            │       │                         │                         │ (cv2.resize()),         │              │
│            │       │                         │                         │ cropping using array    │              │
│            │       │                         │                         │ slicing, and flipping   │              │
│            │       │                         │                         │ images horizontally or  │              │
│            │       │                         │         

### **4.3.5 Display Content Details Course 5**

In [ ]:
choose_and_display_course(cursor)


📚 Available Courses:
1. Introduction to Computer Vision and Image Processing (ID: 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a)
2. Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)
3. Machine Learning Specialization: Mastering the Art of Data Science (ID: 537ad633-a6f1-4e39-bb62-0f5b80726f21)
4. OpenCV Python Tutorial For Beginners (ID: a2ab7375-b89a-48c8-b56e-ce7152b6ec96)
5. Introduction to python (ID: fdd32b6c-cad7-4546-b958-41bf41429b3f)
➡️ Choose a course by number: 2

📖 Displaying details for course: Mastering Natural Language Processing (NLP) with Python (ID: 4c53688f-430a-4d8c-b21c-e796256b46da)


                     📚 Mastering Natural Language Processing (NLP) with Python Course Details                     
╭────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮
│               Property │ Value                                                                                  │
├────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│            Description │ This comprehensive course is designed to take learners from the foundational concepts  │
│                        │ of Natural Language Processing to advanced applications using Python. Through hands-on │
│                        │ lessons and practical examples, you will learn how machines understand, interpret, and │
│                        │ generate human language. Whether you're looking to build chatbots, perform sentiment   │
│                        │ analysis, or dive into language modeling, this course equips you with the skills       │
│                        │ needed to succeed in the world of NLP.                                                 │
│               Category │ NLP                                                                                    │
│                  Level │ Advanced                                                                               │
│                  Price │ $0.0                                                                                   │
│            Total Hours │ 3.3666666666666667 hours                                                               │
│      What You'll Learn │ -       The foundations and core principles of NLP -    Text preprocessing and feature │
│                        │ extraction -     NLP libraries: NLTK, spaCy, Transformers -      Building NLP          │
│                        │ pipelines and models -     Text classification, sentiment analysis, and topic modeling │
│                        │ -   Named entity recognition and part-of-speech tagging -   Sequence models and        │
│                        │ language modeling - Deploying NLP applications in the real world                       │
│ Who This Course Is For │ -       Aspiring data scientists and machine learning engineers -       Developers     │
│                        │ interested in integrating NLP into applications -    Students and researchers          │
│                        │ exploring linguistic data -    Professionals transitioning to AI roles                 │
│          Thumbnail URL │ https://www.google.com/search?sca_esv=b4544fb49b72c327&sxsrf=AHTn8zp3Xv_zJmqKJWI3FU0Q… │
│             Instructor │ Mohamed Hatem                                                                          │
╰────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯

                                            📚 Chapters for this Course                                            
╭──────────┬────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────╮
│ Position │ Chapter ID │ Title                              │ Description                        │ Content Count │
├──────────┼────────────┼────────────────────────────────────┼────────────────────────────────────┼───────────────┤
│        1 │     10     │ Chapter 1: Foundations of NLP      │ This chapter introduces the        │       6       │
│          │            │                                    │ fundamentals of NLP, language      │               │
│          │            │                                    │ structure, and core tools used to  │               │
│          │            │                                    │ manipulate and analyze text data.  │               │
│        2 │     11     │ Chapter 2: Core NLP Tasks and      │ his chapter covers the most widely │       6       │
│          │            │ Models                             │ used tasks and models in NLP,      │               │
│          │            │                                    │ equipping learners to perform      │               │
│          │            │                                    │ real-world language analysis       │               │
│        3 │     12     │ Chapter 3: Advanced NLP and        │ This final chapter introduces      │       6       │
│          │            │ Applications                       │ cutting-edge techniques in NLP and │               │
│          │            │                                    │ how to integrate them into         │               │
│          │            │                                    │ real-world applications.           │               │
╰──────────┴────────────┴────────────────────────────────────┴────────────────────────────────────┴───────────────╯

🔖 Chapter 1: Chapter 1: Foundations of NLP (Position: 1)

📜 Description: This chapter introduces the fundamentals of NLP, language structure, and core tools used to 
manipulate and analyze text data.

...

📜 Description: his chapter covers the most widely used tasks and models in NLP, equipping learners to perform 
real-world language analysis

                                📦 Content for Chapter 2: Core NLP Tasks and Models                                
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     55     │ VIDEO │ https://www.youtube.co… │ Lesson 4 : Word         │ Understand word vector  │ 13.0 minutes │
│            │       │                         │ Embeddings and          │ representations, BERT,  │              │
│            │       │                         │ Transformer Models      │ and transformer-based   │              │
│            │       │                         │                         │ models, and how they    │              │
│            │       │                         │                         │ revolutionized NLP.     │              │
│     56     │  PDF  │        no_url_54        │ Lesson 4 : Word         │ Understand word vector  │ 1.0 minutes  │
│            │       │                         │ Embeddings and          │ representations, BERT,  │              │
│            │       │                         │ Transformer Models pdf  │ and transformer-based   │              │
│            │       │                         │                         │ models, and how they    │              │
│            │       │                         │                         │ revolutionized NLP.     │              │
│     57     │ VIDEO │ https://www.youtube.co… │ Lesson 5 : Sequence     │ Learn to generate text, │ 23.0 minutes │
│            │       │                         │ Modeling and Language   │ translate languages,    │              │
│            │       │                         │ Generation              │ and build               │              │
│            │       │                         │                         │ auto-completion systems │              │
│            │       │                         │                         │ using RNNs and          │              │
│            │       │                         │                         │ Transformers.           │              │
│     58     │  PDF  │        no_url_55        │ Lesson 5 : Sequence     │ Learn to generate text, │ 1.0 minutes  │
│            │       │                         │ Modeling and Language   │ translate languages,    │              │
│            │       │                         │ Generation pdf          │ and build               │              │
│            │       │                         │                         │ auto-completion systems │              │
│            │       │                         │                         │ using RNNs and          │              │
│            │       │                         │                         │ Transformers.           │              │
│     59     │ VIDEO │ https://www.youtube.co… │ Lesson 6: Building and  │ Combine all learned     │ 26.0 minutes │
│            │       │                         │ Deploying NLP           │ concepts to build,      │              │
│            │       │                         │ Applications            │ evaluate, and deploy    │              │
│            │       │                         │                         │ NLP solutions using     │              │
│            │       │                         │                         │ APIs, Flask, and        │              │
│            │       │                         │                         │ Docker.                 │              │
│     60     │  PDF  │        no_url_56        │ Lesson 6: Building and  │ Combine all learned     │ 1.0 minutes  │
│            │       │                         │ Deploying NLP           │ concepts to build,      │              │
│            │       │                         │ Applicat

🔖 Chapter 3: Chapter 3: Advanced NLP and Applications (Position: 3)

📜 Description: This final chapter introduces cutting-edge techniques in NLP and how to integrate them into 
real-world applications.

                              📦 Content for Chapter 3: Advanced NLP and Applications                              
╭────────────┬───────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬──────────────╮
│ Content ID │ Type  │           URL           │ Title                   │ Description             │   Duration   │
├────────────┼───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────┤
│     61     │ VIDEO │ https://www.youtube.co… │ Lesson 7: Word          │ Understand word vector  │ 15.0 minutes │
│            │       │                         │ Embeddings and          │ representations, BERT,  │              │
│            │       │                         │ Transformer Models      │ and transformer-based   │              │
│            │       │                         │                         │ models, and how they    │              │
│            │       │                         │                         │ revolutionized NLP.     │              │
│     62     │  PDF  │        no_url_57        │ Lesson 7: Word          │ Understand word vector  │ 1.0 minutes  │
│            │       │                         │ Embeddings and          │ representations, BERT,  │              │
│            │       │                         │ Transformer Models pdf  │ and transformer-based   │              │
│            │       │                         │                         │ models, and how they    │              │
│            │       │                         │                         │ revolutionized NLP.     │              │
│     63     │ VIDEO │ https://www.youtube.co… │ Lesson 8: Sequence      │ Learn to generate text, │ 36.0 minutes │
│            │       │                         │ Modeling and Language   │ translate languages,    │              │
│            │       │                         │ Generation              │ and build               │              │
│            │       │                         │                         │ auto-completion systems │              │
│            │       │                         │                         │ using RNNs and          │              │
│            │       │                         │                         │ Transformers.           │              │
│     64     │  PDF  │        no_url_58        │ Lesson 8: Sequence      │ Learn to generate text, │ 1.0 minutes  │
│            │       │                         │ Modeling and Language   │ translate languages,    │              │
│            │       │                         │ Generation pdf          │ and build               │              │
│            │       │                         │                         │ auto-completion systems │              │
│            │       │                         │                         │ using RNNs and          │              │
│            │       │                         │                         │ Transformers.           │              │
│     65     │ VIDEO │ https://www.youtube.co… │ Lesson 9: Building and  │ Combine all learned     │ 19.0 minutes │
│            │       │                         │ Deploying NLP           │ concepts to build,      │              │
│            │       │                         │ Applications            │ evaluate, and deploy    │              │
│            │       │                         │                         │ NLP solutions using     │              │
│            │       │                         │                         │ APIs, Flask, and        │              │
│            │       │                         │                         │ Docker.                 │              │
│     66     │  PDF  │        no_url_59        │ Lesson 9: Building and  │ Combine all learned     │ 1.0 minutes  │
│            │       │                         │ Deploying NLP           │ concepts to build,      │              │
│            │       │                         │ Applicat

## **4.4 Assign a user to specific Course**

In [ ]:
def get_students(cursor):
    """Fetch all students from the users table."""
    cursor.execute("SELECT user_ID, full_name FROM users WHERE role = 'student';")
    return cursor.fetchall()

In [ ]:
def get_courses(cursor):
    """Fetch all courses."""
    cursor.execute("SELECT course_ID, course_title FROM courses;")
    return cursor.fetchall()

In [ ]:
def assign_student_to_course(cursor, conn):
    students = get_students(cursor)
    if not students:
        console.print("[bold yellow]⚠️ No students found in the database.[/bold yellow]")
        return

    console.print("\n🎓 [bold underline]Available Students[/bold underline]")
    student_table = Table(title="Students")
    student_table.add_column("No.", style="cyan", justify="right")
    student_table.add_column("Full Name", style="magenta")
    student_table.add_column("User ID", style="dim")

    for idx, (student_id, full_name) in enumerate(students, 1):
        student_table.add_row(str(idx), full_name, student_id)

    console.print(student_table)

    # Select student
    while True:
        choice = input("[Press 'q' to quit] Select a student by number: ").strip().lower()
        if choice == 'q':
            console.print("[bold blue]🚪 Exiting student assignment...[/bold blue]")
            return
        if choice.isdigit() and 1 <= int(choice) <= len(students):
            student_id, student_name = students[int(choice) - 1]
            break
        console.print("[red]❌ Invalid choice. Try again.[/red]")

    # Get Courses
    courses = get_courses(cursor)
    if not courses:
        console.print("[bold yellow]⚠️ No courses found in the database.[/bold yellow]")
        return

    console.print("\n📘 [bold underline]Available Courses[/bold underline]")
    course_table = Table(title="Courses")
    course_table.add_column("No.", style="cyan", justify="right")
    course_table.add_column("Course Title", style="green")
    course_table.add_column("Course ID", style="dim")

    for idx, (course_id, course_title) in enumerate(courses, 1):
        course_table.add_row(str(idx), course_title, course_id)

    console.print(course_table)

    # Select course
    while True:
        choice = input("[Press 'q' to quit] Select a course by number: ").strip().lower()
        if choice == 'q':
            console.print("[bold blue]🚪 Exiting course assignment...[/bold blue]")
            return
        if choice.isdigit() and 1 <= int(choice) <= len(courses):
            course_id, course_title = courses[int(choice) - 1]
            break
        console.print("[red]❌ Invalid choice. Try again.[/red]")

    # Insert into enrollments
    enroll_id = str(uuid4())
    insert_query = '''
        INSERT INTO enrollments (enroll_ID, user_ID, course_ID, status, progress)
        VALUES (%s, %s, %s, 'Active', 0.0)
    '''
    try:
        cursor.execute(insert_query, (enroll_id, student_id, course_id))
        conn.commit()
        console.print(f"[bold green]✅ Student '{student_name}' has been enrolled in '{course_title}'.[/bold green]")
    except Exception as e:
        console.print(f"[bold red]❌ Failed to enroll student: {e}[/bold red]")

In [ ]:
assign_student_to_course(cursor, db_conn)

🎓 Available Students

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 1


📘 Available Courses

                                                      Courses                                                      
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Course Title                                                       ┃ Course ID                            ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Introduction to Computer Vision and Image Processing               │ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │
│   2 │ Mastering Natural Language Processing (NLP) with Python            │ 4c53688f-430a-4d8c-b21c-e796256b46da │
│   3 │ Machine Learning Specialization: Mastering the Art of Data Science │ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │
│   4 │ OpenCV Python Tutorial For Beginners                               │ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │
│   5 │ Introduction to python                                             │ fdd32b6c-cad7-4546-b958-41bf41429b3f │
└─────┴────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a course by number: 5


✅ Student 'Basel Amr Barakat' has been enrolled in 'Introduction to python'.

In [ ]:
assign_student_to_course(cursor, db_conn)

🎓 Available Students

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 1


📘 Available Courses

                                                      Courses                                                      
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Course Title                                                       ┃ Course ID                            ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Introduction to Computer Vision and Image Processing               │ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │
│   2 │ Mastering Natural Language Processing (NLP) with Python            │ 4c53688f-430a-4d8c-b21c-e796256b46da │
│   3 │ Machine Learning Specialization: Mastering the Art of Data Science │ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │
│   4 │ OpenCV Python Tutorial For Beginners                               │ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │
│   5 │ Introduction to python                                             │ fdd32b6c-cad7-4546-b958-41bf41429b3f │
└─────┴────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a course by number: 12


❌ Invalid choice. Try again.

[Press 'q' to quit] Select a course by number: 4


✅ Student 'Basel Amr Barakat' has been enrolled in 'OpenCV Python Tutorial For Beginners'.

## **4.5  See All Course Statuses for a Student**

In [ ]:
def view_student_enrollments(cursor):
    students = get_students(cursor)
    if not students:
        console.print("[bold yellow]⚠️ No students available.[/bold yellow]")
        return

    console.print("\n👨‍🎓 [bold underline]Select a Student[/bold underline]")
    student_table = Table(title="Students")
    student_table.add_column("No.", style="cyan", justify="right")
    student_table.add_column("Full Name", style="magenta")
    student_table.add_column("User ID", style="dim")

    for idx, (student_id, full_name) in enumerate(students, 1):
        student_table.add_row(str(idx), full_name, student_id)

    console.print(student_table)

    while True:
        choice = input("[Press 'q' to quit] Select a student by number: ").strip().lower()
        if choice == 'q':
            console.print("[bold blue]🚪 Exiting enrollment viewer...[/bold blue]")
            return
        if choice.isdigit() and 1 <= int(choice) <= len(students):
            student_id, student_name = students[int(choice) - 1]
            break
        console.print("[red]❌ Invalid choice. Try again.[/red]")

    # Fetch enrollment status
    query = '''
        SELECT c.course_title, e.status, e.progress
        FROM enrollments e
        JOIN courses c ON e.course_ID = c.course_ID
        WHERE e.user_ID = %s
    '''
    cursor.execute(query, (student_id,))
    enrollments = cursor.fetchall()

    if not enrollments:
        console.print(f"[bold yellow]⚠️ '{student_name}' is not enrolled in any courses.[/bold yellow]")
        return

    table = Table(title=f"📘 Enrollments for {student_name}")
    table.add_column("Course", style="green")
    table.add_column("Status", style="cyan")
    table.add_column("Progress (%)", justify="right", style="blue")

    for course_title, status, progress in enrollments:
        table.add_row(course_title, status, f"{progress:.2f}")

    console.print(table)

In [ ]:
view_student_enrollments(cursor)

👨‍🎓 Select a Student

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 1


              📘 Enrollments for Basel Amr Barakat              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Course                               ┃ Status ┃ Progress (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Introduction to python               │ Active │         0.00 │
│ OpenCV Python Tutorial For Beginners │ Active │         0.00 │
└──────────────────────────────────────┴────────┴──────────────┘

In [ ]:
# Try to add new user and test again
add_users(cursor)

👤 Add New User

👤 Full Name:

no name just test


📧 Email:

no mail just test


...

🧪 Preferred Project Types ():

no projects


✅ User no name just test added successfully!

➕ Add another user? [y/n] (n):

n


In [ ]:
view_student_enrollments(cursor)

👨‍🎓 Select a Student

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
│   2 │ no name just test │ a56d5239-f22e-4df1-b611-3510452ea00f │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 2


⚠️ 'no name just test' is not enrolled in any courses.

In [ ]:
assign_student_to_course(cursor, db_conn)

🎓 Available Students

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
│   2 │ no name just test │ a56d5239-f22e-4df1-b611-3510452ea00f │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 2


📘 Available Courses

                                                      Courses                                                      
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Course Title                                                       ┃ Course ID                            ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Introduction to Computer Vision and Image Processing               │ 1fce3a0f-06c3-4809-8bfe-32f7b7ebb75a │
│   2 │ Mastering Natural Language Processing (NLP) with Python            │ 4c53688f-430a-4d8c-b21c-e796256b46da │
│   3 │ Machine Learning Specialization: Mastering the Art of Data Science │ 537ad633-a6f1-4e39-bb62-0f5b80726f21 │
│   4 │ OpenCV Python Tutorial For Beginners                               │ a2ab7375-b89a-48c8-b56e-ce7152b6ec96 │
│   5 │ Introduction to python                                             │ fdd32b6c-cad7-4546-b958-41bf41429b3f │
└─────┴────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a course by number: 1


✅ Student 'no name just test' has been enrolled in 'Introduction to Computer Vision and Image Processing'.

In [ ]:
view_student_enrollments(cursor)

👨‍🎓 Select a Student

                             Students                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Full Name         ┃ User ID                              ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   1 │ Basel Amr Barakat │ 3e6cc479-6c74-4c47-a7bf-fa701d5fc763 │
│   2 │ no name just test │ a56d5239-f22e-4df1-b611-3510452ea00f │
└─────┴───────────────────┴──────────────────────────────────────┘

[Press 'q' to quit] Select a student by number: 2


                      📘 Enrollments for no name just test                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Course                                               ┃ Status ┃ Progress (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Introduction to Computer Vision and Image Processing │ Active │         0.00 │
└──────────────────────────────────────────────────────┴────────┴──────────────┘